# Refining rDNA Sequence
```
pi:ababaian
start: 2017 01 25
complete : 2017 02 13
```
## Introduction

I have ten pilot genomes completed; what I'd like to do is to build a more accurate consensus sequence, in particular with respect to structural variations which were detected.

This may be an iterative process with each alignment cycle producing more structural variants and essentially creating a 'pool' of rDNA sequences which can be aligned to. This pool will then have to 'condensed' in some matter to look at rDNA in general but for now let's worry about making a better rDNA reference.


## Objective

- Make a better rDNA reference sequence; focus on the promoter / transcribed regions

### Secondary Objectives
- In a seperate analysis, build up the inter-transcribed regions. The reference one is poor (with Alu insertions et al.,)

- Include a 5S, align to the reference 5S locus and build up an accurate locus. Again the reference has repeat insertions casually duplicated.

## Materials and Methods

### NA19240 Adjusted Reference
Using IGV, I went though the rDNA sequence with NA19240_pcr loaded and manually adjusted each base such that it matches the consensus/constitutional sequence of rDNA in NA19240.

Sequence range chr13:1,000,000 - 1,013,500 was corrected in this way to make `NA19240_alt.45s.fa`


Fold sequence to width 50
```
cat NA19240_alt.45s.fa | tr '\n' ' ' | sed 's/ //g' - | fold -w 50 > NA19240_alt.45s.2fa
```

### 45S Alternative Haplotypes
Going through the rDNA repeat by hand, there are regions which have indels and/or complex and localized sequence variation. These sequences will require an alternative haplotype to be aligned to and created manually. See Below for regions of possible alternative haplotypes

#### Regions of interest / variation
```
chr13	1000369	1000399	r.ins386CCGG
chr13	1000614	1000653	r.del653
chr13	1000745	1000784	-
chr13	1001651	1001700	r.del1672-1677.T.TT
chr13	1001728	1001777	r.C1749T;r.del1757
chr13	1001789	1001895	altHaplo
chr13	1001919	1001958	r.GT1938CG
chr13	1002052	1002286	altHaplo
chr13	1002287	1002337	r.del2305-2325
chr13	1002463	1002513	altHaplo
chr13	1003525	1003563	-
chr13	1003760	1003849	18s.altHaplo
chr13	1004422	1004559	18s.altHaplo
chr13	1004801	1004937	18s.altHaplo
chr13	1005534	1005589	18s.altHaplo
chr13	1005977	1006205	interTranscribed.altHaplo
chr13	1006513	1006575	pre5.8s.altHaplo
chr13	1007178	1007202	(CGT)7-10
chr13	1007378	1007417	pre28s.AltHaplo.polyC
chr13	1007610	1007614	r.delGCCGC
chr13	1008477	1008576	28s.altHaplo
chr13	1008680	1008739	28s.altHalpo
chr13	1008774	1008819	28s.altHaplo
chr13	1009541	1009654	28s.altHaplo
chr13	1010060	1010173	28s.altHaplo
chr13	1011225	1011299	28s.altHaplo
chr13	1011338	1011407	28s.altHaplo
chr13	1011418	1011420	r.delGCA
chr13	1012049	1012132	28s.altHaplo
chr13	1012962	1013025	end28.altHaplo
chr13	1013433	1013487	end28.altHaplo
```

### Alternative Haplotype Convention

I need a way of represnting haplotypes / structural variants of rDNA.

For a reference sequence `ref` at position `1000` the sequence `{abcdefghi}` has a variants

- `var` is a SNV with sequence `{abcd'e'f'g'h'i'}`
- `ins` is an insertion
- `del` is a deletion

| chr:coord | 1 | 2 | 3 | 4 | 4' | 4'' | 5 | 6 | 7 | 8 | 9 | 10| 11| 12|
|-----------|---|---|---|---|----|-----|---|---|---|---|---|---|---|---|
| ref:10    | a | b | c | d | -  | -   | e | f | g | h | i | j | k | l |
| var:10    | a | b | c | d'| -  | -   | e'| f'| g'| h'| i'| j | k | l |
| ins:10    | a | b | c | d'| d''| d'''| e'| f'| g'| h'| i'| j | k | l |
| del:10    | a | b | c | d'| -  | -   | - | - | - | h'| i'| j | k | l |
| read len  | * | * | * | * | -  | -   | - | - | - | - | - | j | k | l |


#### Haplotype Convention

1. **Anchor Base** is the left-most matching position between the reference and alternative haplotype. {a}
2. **First Variant** is the left-most variant position between the reference and alternative haplotype. {d / d'}
3. Alternative haplotypes and Reference should have the equivalent numerical coordinates at the **Anchor Base**.
4. Alternative haplotypes should be named sequentially: {chrName}_{Anchor Base #}_{uniqueID}
5. The inclusive distance between the Anchor Base and First Variant should be equal to the read-length of the expected sequencing reads. In this toy examples the reads would be 4 bases long.
6. The inclusive distance between the Last Variant {i'} and Last Position {l} should be read length too.

Following these conventions it will be possible to 'collapse' reads into the reference by search and replace. Of course indels won't align 1:1 though.

For each alternative haplotype; also create a bed file entry to 'highlight' the variant region and refer to the alternative chromosome.


Each alternative haplotype should be a single continuous contig; phasing information (for now) is not included but can be accounted for.

If there is a distance greater then Read Length between two variants, then they should be placed on seperate haplotypes. This prevents mis-mapping to duplicate reference sequences in these 'in-between spaces' such as `{efghij}` in the example below.

| chr:coord | 1 | 2 | 3 | 4     | 5 | 6 | 7 | 8 | 9 | 10| 11    | 12| 13| 14|
|-----------|---|---|---|-------|---|---|---|---|---|---|-------|---|---|---|
| ref:10    | a | b | c | d     | e | f | g | h | i | j | k     | l | m | n |
| varA:10   | a | b | c | **d'**| e | f | g | - | - | - | -     | - | - | - |
| varB:10   | - | - | - | -     | - | - | - | h | i | j |**k'** | l | m | n |
| read len  |   |   |   |       | * | * | * | * |   |   |       |   |   |   ||


##### Example Alternative Haplotypes
```
>chrR
ABCDEFGHI
abcdefghi
jklmnopqr

>chrR_10_var
NNNNNNNNN
abcdefghi
jklNNNNNN

>chrR_10_ins
NNNNNNNNN
abcdddefg
hijklNNNN

>chrR_10_del
NNNNNNNNN
abcdhijkl
NNNNNNNNN
```

```
chrR  10  19  chrR_10_var
chrR  10  19  chrR_10_ins
chrR  10  19  chrR_10_del
``` 

### pre 45S Sequence

The sequence I'm using are for the 45S transcribed portion of rDNA; expand the sequence at least 1 kb upstream to include some of the 'promoter' region and possibly bifurcations/ haplotypes which exist there.


#### Extract reads with unpaired mates
After doing a primary go-through. Look for read pairs with unmapped mates and align the mates to themselves to 'build up' alternative haplotypes which were not possible to align. Larger structural differences.

```
samtools view -H NA19240_pcr.bam > align.header.tmp
samtools view -b -f 8 NA19240_pcr.bam > NA19240.f8.tmp
samtools sort ... > NA19240.f8.bam
```
```
>Softclipped_alignf8_pre18S
GTCGCCCGCCCTGCTGGGCCCATCCTCAAGACTGGGAAGCCGGGCTCATCATCGTGAGGCCCGGAGGCCGCTTTC

>Softclipped_alignf8_4935-
AGATCAATTTGTCCTCTTTTGTAATAAAAAAAAAAAGTCTTTAAAAAAAGATTT

>Softclipped_alignf8_5036+
TAAATGTCAAATATAACAAAAAATGCACAGGTATTAACATACTGAATAGGGAAATGCTGTAAGATTTTACTCTAAGAG

```


```
>Softclipped_alignf8_pre18S
GTCGCCCGCCCTGCTGGGCCCATCCTCAAGACTGGGAAGCCGGGCTCATCATCGTGAGGCCCGGAGGCCGCTTTC

Aligned to chr16:33962418-33962492

00000001 gtcgcccgccctgctgggcccatcctcaagactgggaagccgggctcatc 00000050
>>>>>>>> |||||||||||||| | ||||||||||  |||||||| |||||||||||| >>>>>>>>
33962418 gtcgcccgccctgccgcgcccatcctcgcgactgggacgccgggctcatc 33962467

00000051 atcgtgaggcccggaggccgctttc 00000075
>>>>>>>>  ||| |||||| ||||||||||||| >>>>>>>>
33962468 ctcgcgaggcctggaggccgctttc 33962492
```

These reads likely represent 'forks' where the pseudo-rRNA or something similar is inserted in  to the rest of the genome. Or there are TE insertions within the rDNA arrays. This may be interesting but for now focus on the main rDNA sequence and obvious haplotypes.


### NR_046235.1 Updated Sequence

Blast searching for hits from alternative haplotype I found an updated 45S rRNA sequence published earlier in 2016. 
NR_046235.1.

Interestingly a large amount of the consensus corrections (but not all) from NA19240 are in common with NR046235 and different from the original reference (U....). Wish I knew who was updating the sequence to chat with them.


## 5S Sequence + Promoter

Analysis to this point has focused on 45S sequence. I'd like to do the same for 5S and the promoter

see: [5S Sequence](./20170202_5S_subunit.ipynb)

see: [Promoter Sequence](../0170211_rRNA_promoter.ipynb)

## Results / Discussion


### NA19240 Variant Sequence

Alignment to reference sequence


#### Alignment to reference

```
CLUSTAL 2.1 multiple sequence alignment


reference_rDNA      GCTGACACGCTGTCCTCTGGCGACCTGTCGTCGGAGAGGTTGGGCCTCCGGATGCGCGCG
NA19240_45S         GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCCTCCGGATGCGCGCG
                    ******************************  ****************************

reference_rDNA      GGGCTCTGGCCTCACGGTGACCGG-CTAGCCGGCCGCGCTCCTGCCTTGAGCCGCCTGCC
NA19240_45S         GGGCTCTGGCCTACCGGTGACCCGGCTAGCCGGCCGCGCTCCTG-CTTGAGCCGCCTGCC
                    ************..******** * ******************* ***************

reference_rDNA      GCGGCCCGCGGGCCTGCTGTTCTCTCGCGCGTCCGAGCGTCCCGACTCCCGGTGCCGGCC
NA19240_45S         GGGGCCCGCGGGCCTGCTGTTCTCTCGCGCGTCCGAGCGTCCCGACTCCCGGTGCCGGCC
                    * **********************************************************

reference_rDNA      CGGGTCCGGGTCTCTGACCCACCCGGGGGCGGCGGGGAAGGCGGCGAGGGCCACCGTG-C
NA19240_45S         CGGGTCCGGGTCTCTGACCCACCCGGGGGCGGCGGGGAAGGCGGCGAGGGCCACCGTGCC
                    ********************************************************** *

reference_rDNA      CCCGTGCGCTCTCCGCTGCGGGCGCCCGGGGCG-CCGC-ACAACCCCACCCG-CTGGCTC
NA19240_45S         CCCGTGCGCTCTCCGCTGCGGGCGCCCGGGGCGGCCGCGACAACCCCACCCCGCTGGCTC
                    ********************************* **** ************  *******

reference_rDNA      CGTGCCGTGCGTGTCAGGCGTTCTCGTCTCCGCGGGGTTGTCCGCCGCCCCTTCCCCGGA
NA19240_45S         CGTGCCGTGCGTGTCAGGCGTTCTCGTCTCCGCGGGGTTGTCCGCCGCCCCTTCCCCGGA
                    ************************************************************

reference_rDNA      GTGGGGGG-TGGCCGGAGCCGATCGGCTCGCTGGCCGGCCGGCCTCCGCTCCCGGGGGGC
NA19240_45S         GTGGGGGGTTGGCCGGAGCCGATCGGCTCGCTGGCCGGCCGGCCTCCGCTCCCGGGGGGC
                    ******** ***************************************************

reference_rDNA      TCTTCG--ATCGATGTGGTGACGTCGTGCTCTCCCGGGCCGGGTCCGAGCCGCGACGGGC
NA19240_45S         TCTTCGTGATCGATGTGGTGACGTCGTGCTCTCCCGGGCCGGGTCCGAGCCGCGACGGGC
                    ******  ****************************************************

reference_rDNA      GAGGGGCGGACGTTCGTGGCGAACGGGACCGTCCTTCTCGCTCCGCCCG--CGCGGTCCC
NA19240_45S         GAGGGGCGGACGTTCGTGGCGAACGGGACCGTCCTTCTCGCTCCGCCCCGCTGGGGTCCC
                    ************************************************    * ******

reference_rDNA      CTCGTCTGCTCCTCTCCCCGCCCGCCGGCCG-GCGTGTGGGAAGGCGTGGGGTGCGGACC
NA19240_45S         CTCGTCT-CTCCTCTCCCCGCCCGCCGGCGGTGCGTGTGGGAAGGCGTGGGGTGCGGACC
                    ******* ********************* * ****************************

reference_rDNA      CCGGCCCGACCTCGCCGTCCCGCCCGCCGCCTTCG-CTTCGCGGGTGCGGGCCGGCGGGG
NA19240_45S         CCGGCCCGACCTCGCCGTCCCGCCCGCCGCCTTCTGCGTCGCGGGTGCGGGCCGGCGGGG
                    **********************************  * **********************

reference_rDNA      TCCTCTGACGCGGCAGACAGCCCTGCCTGTCGCCTCCAGTGGTTGTCGACTTGCGGGCGG
NA19240_45S         TCCTCTGACGCGGCAGACAGCCCTCGCTGTCGCCTCCAGTGGTTGTCGACTTGCGGGCGG
                    ************************  **********************************

reference_rDNA      CCCCCCTCCGCGGCGGTGGGGGTGCCGTCCCGCCGGCCCGTCGTGCTGCCCTCTCGGGGG
NA19240_45S         CCCCCCTCCGCGGCGGTGGGGGTGCCGTCCCGCCGGCCCGTCGTGCTGCCCTCTCGGGGG
                    ************************************************************

reference_rDNA      GGGTTTGCGCGAGCGTCGGCTCCGCCTGGGCCCTTGCGGTGCTCCTGGAGCGCTCCGGGT
NA19240_45S         G--TTTGCGCGAGCGTCGGCTCCGCCTGGGCCCTTGCGGTGCTCCTGGAGCGCTCCGGGT
                    *  *********************************************************

reference_rDNA      TGTCCCTCAGGTGCCCGAGGCCGAACGGTGGTGTGTCGTTCCCGCCCCCGGCGCCCCCTC
NA19240_45S         TGTCCCTCAGGTGCCCGAGGCCGAACGGTGGTGTGTCGTTCCCGCCCCCGGCGCCCCCTC
                    ************************************************************

reference_rDNA      CTCCGGTCGCCGCCGCGGTGTCCGCGCGTGGGTCCTGAGGGAGCTCGTCGGTGTGGGGTT
NA19240_45S         CTCCGGTCGCCGCCGCGGTGTCCGCGCGTGGGTCCTGAGGGAGCTCGTCGGTGTGGGGTT
                    ************************************************************

reference_rDNA      CGAGGCGGTTTGAGTGAGACGAGACGAGACGCGCCCCTCCCACGCGGGGAAGGGCGCCCG
NA19240_45S         CGAGGCGGTTTGAGTGAGACGAGACGAGACGCGCCCCTCCCACGCGGGGAAGGGCGCCCG
                    ************************************************************

reference_rDNA      CCTGCTCTCGGTGAGCGCACGTCCCGTGCTCCCCTCTGGCGGGTGCGCGCGGGCCGTGTG
NA19240_45S         CCTGCTCTCGGTGAGCGCACGTCCCGTGCTCCCCTCTGGCGGGTGCGCGCGGGCCGTGTG
                    ************************************************************

reference_rDNA      AGCGATCGCGGTGGGTTCGGGCCGGTGTGACGCGTGCGCCGGCCGGCCGCCGAGGGGCTG
NA19240_45S         AGCGATCGCGGTGGGTTCGGGCCGGTGTGACGCGTGCGCCGGCCGGCCGCCGAGGGGCTG
                    ************************************************************

reference_rDNA      CCGTTCTGCCTCCGACCGGTCGTGTGTGGGTTGACTTCGGAGGCGCTCTGCCTCGGAAGG
NA19240_45S         CCGTTCTGCCTCCGACCGGTCGTGTGTGGGTTGACTTCGGAGGCGCTCTGCCTCGGAAGG
                    ************************************************************

reference_rDNA      AAGGAGGTGGGTGGACGGGGGGGCCTGGTGGGGTTGCGCGCACGCGCGCACCGGCCGGGC
NA19240_45S         AAGGAGGTGGGTGGACGGGGGGGCCTGGTGGGGTTGCGCGCACGCGCGCACCGGCCGGGC
                    ************************************************************

reference_rDNA      CCCCGCCCTGAACGCGAACGCTCGAGGTGGCCGCGCGCAGGTGTTTCCTCGTACCGCAGG
NA19240_45S         CCCCGCCCTGAACGCGAACGCTCGAGGTGGCCGCGCGCAGGTGTTTCCTCGTACCGCAGG
                    ************************************************************

reference_rDNA      GCCCCCTCCCTTCCCCAGGCGTCCCTCGGCGCCTCTGCGGGCCCGAGGAGGAGCGGCTGG
NA19240_45S         GCCCCCTCCCTTCCCCAGGCGTCCCTCGGCGCCTCTGCGGGCCCGAGGAGGAGCGGCTGG
                    ************************************************************

reference_rDNA      CGGGTGGGGGGAGTGTGACCCACCCTCGGTGAGAAAAGCCTTCTCTAGCGATCTGAGAGG
NA19240_45S         CGGGTGGGGGGAGTGTGACCCACCCTCGGTGAGAAAAGCCTTCTCTAGCGATCTGAGAGG
                    ************************************************************

reference_rDNA      CGTGCCTTGGGGGTACCGGATCCCCCGGGCCGCCGCCTCTGTCTCTGCCTCCGTTATGGT
NA19240_45S         CGTGCCTTGGGGGTACCGGATCCCCCGGGCCGCCGCCTCTGTCTCTGCCTCCGTTATGGT
                    ************************************************************

reference_rDNA      AGCGCTGCCGTAGCGACCCGCTCGCAGAGGACCCTCCTCCGCTTCCCCCTCGACGGGGTT
NA19240_45S         AGCGCTGCCGTAGCGACCCGCTCGCAGAGGACCCTCCTCCGCTTCCCCCTCGACGGGGTT
                    ************************************************************

reference_rDNA      GGGGGGGAGAAGCGAGGGTTCCGCCGGCCACCGCGGTGGTGGCCGAGTGCGGCTCGTCGC
NA19240_45S         GGGGGGGAGAAGCGAGGGTTCCGCCGGCCACCGCGGTGGTGGCCGAGTGCGGCTCGTCGC
                    ************************************************************

reference_rDNA      CTACTGTGGCCCGCGCCTCCCCCTTCCGAGTCGGGGGAGGATCCCGCCGGGCCGGGCCCG
NA19240_45S         CTACTGTGGCCCGCGCCTCCCCCTTCCGAGTCGGGGGAGGATCCCGCCGGGCCGGGCCCG
                    ************************************************************

reference_rDNA      GCGCTCCCACCCAGCGGGTTGGGACGCGGCGGCCGGCGGGCGGTGGGTGTGCGCGCCCGG
NA19240_45S         GCGTT----CCCAGCGGGTTGGGACGCGGCGGCCGGCGGGCGGTGGGTGTGCGCGCCCGG
                    *** *    ***************************************************

reference_rDNA      CGCTCTGTCCGGCGCGTGACCCCCTCCGTCCGCGAGTCGGCTCTCCGCCCGCTCCCGTGC
NA19240_45S         CGCTCTGTCCGGCGCGTGACCCCCTCCG-CCGCGAGTCGGCTCTCCGCCCGCTCCCGTGC
                    **************************** *******************************

reference_rDNA      CGAGTCGTGACCGGTGCCGACGACCGCGTTTGCGTGGCACGGGGTCGGGCCCGCCTGGCC
NA19240_45S         CGAGTCGTGACCGGTGCCGACGACCGCGTTTGCGTGGCACGGGGTCGGGCCCGCCTGGCC
                    ************************************************************

reference_rDNA      CTGGGAAAGCGTCCCACGGTGGGGGCGCGCCGGTCTCCCGGAGCGGGACCGGGTCGGAGG
NA19240_45S         CTGGGAAAGCGTCCCACGGTGGGGGCGCGCCGGTCTCCCGGAGCGGGACCGGGTCGGAGG
                    ************************************************************

reference_rDNA      ATGGACGAGAATCACGAGCGACGGTGGTGGTGGCGTGTCGGGTTCGTGGCTGCGGTCGCT
NA19240_45S         ATGGACGAGAATCACGAGCGACGGTGGTGGTGGCGTGTCGGGTTCGTGGCTGCGGTCGCT
                    ************************************************************

reference_rDNA      CCGGGGCCCCCGGTGGCGGGGCCCCGGGGCTCGCGAGGCGGTTCTCGGTGGGGGCCGAGG
NA19240_45S         CCGGGGCCCCCGGTGGCGGGGCCCCGGGGCTCGCGAGGCGGTTCTCGGTGGGGGCCGAGG
                    ************************************************************

reference_rDNA      GCCGTCCGGCGTCCCAGGCGGGGCGCCGCGGGACCGCCCTCGTGTCTGTGGCGGTGGGAT
NA19240_45S         GCCGTCCGGCGTCCCAGGCGGGGCGCCGCGGGACCGCCCTCGTGTCTGTGGCGGTGGGAT
                    ************************************************************

reference_rDNA      CCCGCGGCCGTGTTTTCCTGGTGGCCCGGCCGTGCCTGAGGTTTCTCCCCGAGCCGCCGC
NA19240_45S         CCCGCGGCCGTGTTTTCCTGGTGGCCCGGCCGTGCCTGAGGTTTCTCCCCGAGCCGCCGC
                    ************************************************************

reference_rDNA      CTCTGCGGGCTCCCGGGTGCCCTTGCCCTCGCGGTCCCCGGCCCTCGCCCGTCTGTGCCC
NA19240_45S         CTCTGCGGGCTCCCGGGTGCCCTTGCCCTCGCGGTCCCCGGCCCTCGCCCGTCTGTGCCC
                    ************************************************************

reference_rDNA      TCTTCCCCGCCCGCCGCCCGCCGATCCTCTTCTTCCCCCCGAGCGGCTCACCGGCTTCAC
NA19240_45S         TCTTCCCCGCCCGCCGCCCGCCGATCCTCTTCTTCCCCCCGAGCGGCTCACCGGCTTCAC
                    ************************************************************

reference_rDNA      GTCCGTTGGTGGCCCCGCCTGGGACCGAACCCGGCACCGCCTCGTGGGGCGCCGCCGCCG
NA19240_45S         GTCCGTTGGTGGCCCCGCCTGGGACCGAACCCGGCACCGCCTCGTGGGGCGCCGCCGCCG
                    ************************************************************

reference_rDNA      GCCACTGATCGGCCCGGCGTCCGCGTCCCCCGGCGCGCGCCTTGGGGACCGGGTCGGTGG
NA19240_45S         GCCACTGATCGGCCCGGCGTCCGCGTCCCCCGGCGCGCGCCTTGGGGACCGGGTCGGTGG
                    ************************************************************

reference_rDNA      CGCGCCGCGTGGGGCCCGGTGGGCTTCCCGGAGGGTTCCGGGGGTCGGCCTGCGGCGCGT
NA19240_45S         CGCCCCGCGTGGGGCCCGGTGGGCTTCCCGGAGGGTTCCGGGGGTCGGCCTGCGGCGCGT
                    *** ********************************************************

reference_rDNA      GCGGGGGAGGAGACGGTTCCGGGGGACCGGCCGCGGCTGCGGCGGCGGCGGTGGTGGGGG
NA19240_45S         GCGGGGGAGGAGACGGTTCCGGGGGACCGGCCGCGACTGCGGCG---GCGGTGGTGGGGG
                    ***********************************.********   *************

reference_rDNA      GAGCCGCGGGGATCGCCGAGGGCCGGTCGGCCGCCCCGGGTGCCCCGCGGTGCCGCCGGC
NA19240_45S         GAGCCGCGGGGATCGCCGAGGGCCGGTCGGCCGCCCCGGGTGCCGCGCGGTGCCGCCGGC
                    ******************************************** ***************

reference_rDNA      GGCGGTGAGGCCCCGCGCGTGTGTCCCGGCTGCGGTCGGCCGCGCTCGAGGGGTCCCCGT
NA19240_45S         GGCGGTGAGGCCCCGCGCGTGTGTCCCGGCTGCGGTCGGCCGCGCTCGAGGGGTCCCCGT
                    ************************************************************

reference_rDNA      GGCGTCCCCTTCCCCGCCGGCCGCCTTTCTCGCGCCTTCCCCGTCGCCCCGGCCTCGCCC
NA19240_45S         GGCGTCCCCTTCCCCGCCGGCCGCCTTTCTCGCGCCTTCCCCGTCGCCCCGGCCTCGCCC
                    ************************************************************

reference_rDNA      GTGGTCTCTCGTCTTCTCCCGGCCCGCTCTTCCGAACCGGGTCGGCGCGTCCCCCGGGTG
NA19240_45S         GTGGTCTCTCGTCTTCTCCCGGCCCGCTCTTCCGAACCGGGTCGGCGCGTCCCCCGGGTG
                    ************************************************************

reference_rDNA      CGCCTCGCTTCCCGGGCCTGCCGCGGCCCTTCCCCGAGGCGTCCGTCCCGGGCGTCGGCG
NA19240_45S         CGCCTCGCTTCCCGGGCCTGCCGCGGCCCTTCCCCGAGGCGTCCGTCCCGGGCGTCGGCG
                    ************************************************************

reference_rDNA      TCGGGGAGAGCCCGTCCTCCCCGCGTGGCGTCGCCCCGTTCGGCGCGCGCGTGCGCCCGA
NA19240_45S         TCGGGGAGAGCCCGTCCTCCCCGCGTGGCGTCGCCCCGTTCGGCGCGCGCGTGCGCCCGA
                    ************************************************************

reference_rDNA      GCGCGGCCCGGTGGTCCCTCCCGGACAGGCGTTCGTGCGACGTGTGGCGTGGGTCGACCT
NA19240_45S         GCGCGGCCCGGTGGTCCCTCCCGGACAGGCGTTCGTGCGACGTGTGGCGTGGGTCGACCT
                    ************************************************************

reference_rDNA      CCGCCTTGCCGGTCGCTCGCCCTCTCCCCGGGTCGGGGGGTGGGGCCCGGGCCGGGGCCT
NA19240_45S         CCGCCTTGCCGGTCGCTCGCCCTCTCCCCGGGTCGGGGGGTGGGGCCCGGGCCGGGGCCT
                    ************************************************************

reference_rDNA      CGGCCCCGGTCGCTGCCTCCCGTCCCGGGCGGGGGCGGGCGCGCCGGCCGGCCTCGGTCG
NA19240_45S         CGGCCCCGGTCGCGGTCCCCCGTCCCGGGCGGGGGCGGGCGCGCCGGCCGGCCTCGGTCG
                    ************* * * ******************************************

reference_rDNA      -CCCTCCCTTGGCCGTCGTGTGGCGTGTGCCACCCCTGCGCCGGCGCCCGCCGGCGGGGC
NA19240_45S         GCCCTCCCTTGGCCGTCGTGTGGCGTGTGCCACCCCTGCGCCCGCGCCCGCCGGCGGGGC
                     ***************************************** *****************

reference_rDNA      TCGGAGCCGGGCTTCGGCCGGGCCCCGGGCCCTCGACCGGACCGGCTGCGCGGGCGCTGC
NA19240_45S         TCGGAGCCGGGCTTCGGCCGGGCCCCGGGCCCTCGACCGGACCGG-TGCGCGGGCGCTGC
                    ********************************************* **************

reference_rDNA      GGCCGCACGGCGCGACTGTCCCCGGGCCGGGCACCGCGGTCCGCCTCTCGCTCGCCGCCC
NA19240_45S         GGCCGCACGGCGCGACTGTCCCCGGGCCGGGCACCGCGGTCCGCCTCTCGCTCGCCGCCC
                    ************************************************************

reference_rDNA      GGACGTCGGGGCCGCCCCGCGGGGCGGGCGGAGCGCCGTCCCCGCCTCGCCGCCGCCCGC
NA19240_45S         GGACGTCGGGGCCGCCCCGCGGGGCGGGCGGAGCGCCGTCCCCGCCTCGCCGCCGCCCGC
                    ************************************************************

reference_rDNA      GGGCGCCGGCCGCGCGCGCGCGCGCGTGGCCGCCGGTCCCTCCCGGCCGCCGGGCGCGGG
NA19240_45S         GGGCGCCGGCCGCGCGCGCGCGCG--TGGCCGCCGGTCCCTCCCGGCCGCCGGGCGCGGG
                    ************************  **********************************

reference_rDNA      TCGGGCCGTCCGCCTCCTCGCGGGCGGGCGCGACGAAGAAGCGTCGCGGGTCTGTGGCGC
NA19240_45S         TCGGGCCGTCCGCCTCCTCGCGGGCGGGCGCGACGAAGAAGCGTCGCGGGTCTGTGGCGC
                    ************************************************************

reference_rDNA      GGGGCCCCCGGTGGTCGTGTCGCGTGGGGGGCGGGTGGTTGGGGCGTCCGGTTCGCCGCG
NA19240_45S         GGGGCCCCGG-TGGTCGTGTCGCGTGGGGGGCGGGTGGTTGGGGCGTCCGGTTCGCCGCG
                    ******** * *************************************************

reference_rDNA      CCCCGCCCCGGCCCCACCGGTCCCGGCCGCCGCCCCCGCGCCCGCTCGCTCCCTCCCGTC
NA19240_45S         CCCCGCCCCGGCCCCACCGGTCCCGGCCGCCGCCCCCGCGCCCGCTCGCTCCCTCCCGTC
                    ************************************************************

reference_rDNA      CGCCCGTCCGCGGCCCGTCCGTCCGTCCGTCCGTCGTCCTCCTCGCTTGCGGGGCGCCGG
NA19240_45S         CGCCCGTCCGCGGCCCGTCCGTCCGTCCGTCCGTCGTCCTCCTCGCTTGCGGGGCGCCGG
                    ************************************************************

reference_rDNA      GCCCGTCCTCGCGAGGCCCCCCGGCCGGCCGTCCGGCCGCGTCGGGGGCTCGCCGCGCTC
NA19240_45S         GCCCGTCCTCGCGAGGCCCCCCGGCCGGCCGTCCGGCCGCGTCGGGGCCTCGCCGCGCTC
                    *********************************************** ************

reference_rDNA      TACCTTACCTACCTGGTTGATCCTGCCAGTAGCATATGCTTGTCTCAAAGATTAAGCCAT
NA19240_45S         TACCTTACCTACCTGGTTGATCCTGCCAGTAGCATATGCTTGTCTCAAAGATTAAGCCAT
                    ************************************************************

reference_rDNA      GCATGTCTAAGTACGCACGGCCGGTACAGTGAAACTGCGAATGGCTCATTAAATCAGTTA
NA19240_45S         GCATGTCTAAGTACGCACGGCCGGTACAGTGAAACTGCGAATGGCTCATTAAATCAGTTA
                    ************************************************************

reference_rDNA      TGGTTCCTTTGGTCGCTCGCTCCTCTCCTACTTGGATAACTGTGGTAATTCTAGAGCTAA
NA19240_45S         TGGTTCCTTTGGTCGCTCGCTCCTCTCCTACTTGGATAACTGTGGTAATTCTAGAGCTAA
                    ************************************************************

reference_rDNA      TACATGCCGACGGGCGCTGACCCCCTTCGCGGGGGGGATGCGTGCATTTATCAGATCAAA
NA19240_45S         TACATGCCGACGGGCGCTGACCCCCTTCGCGGGGGGGATGCGTGCATTTATCAGATCAAA
                    ************************************************************

reference_rDNA      ACCAACCCGGTCAGCCCCTCTCCGGCCCCGGCCGGGGGGCGGGCGCCGGCGGCTTTGGTG
NA19240_45S         ACCAACCCGGTCAGCCCCTCTCCGGCCCCGGCCGGGGGGCGGGCGCCGGCGGCTTTGGTG
                    ************************************************************

reference_rDNA      ACTCTAGATAACCTCGGGCCGATCGCACGCCCCCCGTGGCGGCGACGACCCATTCGAACG
NA19240_45S         ACTCTAGATAACCTCGGGCCGATCGCACGCCCCCCGTGGCGGCGACGACCCATTCGAACG
                    ************************************************************

reference_rDNA      TCTGCCCTATCAACTTTCGATGGTAGTCGCCGTGCCTACCATGGTGACCACGGGTGACGG
NA19240_45S         TCTGCCCTATCAACTTTCGATGGTAGTCGCCGTGCCTACCATGGTGACCACGGGTGACGG
                    ************************************************************

reference_rDNA      GGAATCAGGGTTCGATTCCGGAGAGGGAGCCTGAGAAACGGCTACCACATCCAAGGAAGG
NA19240_45S         GGAATCAGGGTTCGATTCCGGAGAGGGAGCCTGAGAAACGGCTACCACATCCAAGGAAGG
                    ************************************************************

reference_rDNA      CAGCAGGCGCGCAAATTACCCACTCCCGACCCGGGGAGGTAGTGACGAAAAATAACAATA
NA19240_45S         CAGCAGGCGCGCAAATTACCCACTCCCGACCCGGGGAGGTAGTGACGAAAAATAACAATA
                    ************************************************************

reference_rDNA      CAGGACTCTTTCGAGGCCCTGTAATTGGAATGAGTCCACTTTAAATCCTTTAACGAGGAT
NA19240_45S         CAGGACTCTTTCGAGGCCCTGTAATTGGAATGAGTCCACTTTAAATCCTTTAACGAGGAT
                    ************************************************************

reference_rDNA      CCATTGGAGGGCAAGTCTGGTGCCAGCAGCCGCGGTAATTCCAGCTCCAATAGCGTATAT
NA19240_45S         CCATTGGAGGGCAAGTCTGGTGCCAGCAGCCGCGGTAATTCCAGCTCCAATAGCGTATAT
                    ************************************************************

reference_rDNA      TAAAGTTGCTGCAGTTAAAAAGCTCGTAGTTGGATCTTGGGAGCGGGCGGGCGGTCCGCC
NA19240_45S         TAAAGTTGCTGCAGTTAAAAAGCTCGTAGTTGGATCTTGGGAGCGGGCGGGCGGTCCGCC
                    ************************************************************

reference_rDNA      GCGAGGCGAGCCACCGCCCGTCCCCGCCCCTTGCCTCTCGGCGCCCCCTCGATGCTCTTA
NA19240_45S         GCGAGGCGAGCCACCGCCCGTCCCCGCCCCTTGCCTCTCGGCGCCCCCTCGATGCTCTTA
                    ************************************************************

reference_rDNA      GCTGAGTGTCCCGCGGGGCCCGAAGCGTTTACTTTGAAAAAATTAGAGTGTTCAAAGCAG
NA19240_45S         GCTGAGTGTCCCGCGGGGCCCGAAGCGTTTACTTTGAAAAAATTAGAGTGTTCAAAGCAG
                    ************************************************************

reference_rDNA      GCCCGAGCCGCCTGGATACCGCAGCTAGGAATAATGGAATAGGACCGCGGTTCTATTTTG
NA19240_45S         GCCCGAGCCGCCTGGATACCGCAGCTAGGAATAATGGAATAGGACCGCGGTTCTATTTTG
                    ************************************************************

reference_rDNA      TTGGTTTTCGGAACTGAGGCCATGATTAAGAGGGACGGCCGGGGGCATTCGTATTGCGCC
NA19240_45S         TTGGTTTTCGGAACTGAGGCCATGATTAAGAGGGACGGCCGGGGGCATTCGTATTGCGCC
                    ************************************************************

reference_rDNA      GCTAGAGGTGAAATTCTTGGACCGGCGCAAGACGGACCAGAGCGAAAGCATTTGCCAAGA
NA19240_45S         GCTAGAGGTGAAATTCTTGGACCGGCGCAAGACGGACCAGAGCGAAAGCATTTGCCAAGA
                    ************************************************************

reference_rDNA      ATGTTTTCATTAATCAAGAACGAAAGTCGGAGGTTCGAAGACGATCAGATACCGTCGTAG
NA19240_45S         ATGTTTTCATTAATCAAGAACGAAAGTCGGAGGTTCGAAGACGATCAGATACCGTCGTAG
                    ************************************************************

reference_rDNA      TTCCGACCATAAACGATGCCGACCGGCGATGCGGCGGCGTTATTCCCATGACCCGCCGGG
NA19240_45S         TTCCGACCATAAACGATGCCGACCGGCGATGCGGCGGCGTTATTCCCATGACCCGCCGGG
                    ************************************************************

reference_rDNA      CAGCTTCCGGGAAACCAAAGTCTTTGGGTTCCGGGGGGAGTATGGTTGCAAAGCTGAAAC
NA19240_45S         CAGCTTCCGGGAAACCAAAGTCTTTGGGTTCCGGGGGGAGTATGGTTGCAAAGCTGAAAC
                    ************************************************************

reference_rDNA      TTAAAGGAATTGACGGAAGGGCACCACCAGGAGTGGAGCCTGCGGCTTAATTTGACTCAA
NA19240_45S         TTAAAGGAATTGACGGAAGGGCACCACCAGGAGTGGAGCCTGCGGCTTAATTTGACTCAA
                    ************************************************************

reference_rDNA      CACGGGAAACCTCACCCGGCCCGGACACGGACAGGATTGACAGATTGATAGCTCTTTCTC
NA19240_45S         CACGGGAAACCTCACCCGGCCCGGACACGGACAGGATTGACAGATTGATAGCTCTTTCTC
                    ************************************************************

reference_rDNA      GATTCCGTGGGTGGTGGTGCATGGCCGTTCTTAGTTGGTGGAGCGATTTGTCTGGTTAAT
NA19240_45S         GATTCCGTGGGTGGTGGTGCATGGCCGTTCTTAGTTGGTGGAGCGATTTGTCTGGTTAAT
                    ************************************************************

reference_rDNA      TCCGATAACGAACGAGACTCTGGCATGCTAACTAGTTACGCGACCCCCGAGCGGTCGGCG
NA19240_45S         TCCGATAACGAACGAGACTCTGGCATGCTAACTAGTTACGCGACCCCCGAGCGGTCGGCG
                    ************************************************************

reference_rDNA      TCCCCCAACTTCTTAGAGGGACAAGTGGCGTTCAGCCACCCGAGATTGAGCAATAACAGG
NA19240_45S         TCCCCCAACTTCTTAGAGGGACAAGTGGCGTTCAGCCACCCGAGATTGAGCAATAACAGG
                    ************************************************************

reference_rDNA      TCTGTGATGCCCTTAGATGTCCGGGGCTGCACGCGCGCTACACTGACTGGCTCAGCGTGT
NA19240_45S         TCTGTGATGCCCTTAGATGTCCGGGGCTGCACGCGCGCTACACTGACTGGCTCAGCGTGT
                    ************************************************************

reference_rDNA      GCCTACCCTACGCCGGCAGGCGCGGGTAACCCGTTGAACCCCATTCGTGATGGGGATCGG
NA19240_45S         GCCTACCCTACGCCGGCAGGCGCGGGTAACCCGTTGAACCCCATTCGTGATGGGGATCGG
                    ************************************************************

reference_rDNA      GGATTGCAATTATTCCCCATGAACGAGGGAATTCCCGAGTAAGTGCGGGTCATAAGCTTG
NA19240_45S         GGATTGCAATTATTCCCCATGAACGAGG-AATTCCC-AGTAAGTGCGGGTCATAAGCTTG
                    **************************** ******* ***********************

reference_rDNA      CGTTGATTAAGTCCCTGCCCTTTGTACACACCGCCCGTCGCTACTACCGATTGGATGGTT
NA19240_45S         CGTTGATTAAGTCCCTGCCCTTTGTACACACCGCCCGTCGCTACTACCGATTGGATGGTT
                    ************************************************************

reference_rDNA      TAGTGAGGCCCTCGGATCGGCCCCGCCGGGGTCGGCCCACGGCCCTGGCGGAGCGCTGAG
NA19240_45S         TAGTGAGGCCCTCGGATCGGCCCCGCCGGGGTCGGCCCACGGCCCTGGCGGAGCGCTGAG
                    ************************************************************

reference_rDNA      AAGACGGTCGAACTTGACTATCTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGTG
NA19240_45S         AAGACGGTCGAACTTGACTATCTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGTG
                    ************************************************************

reference_rDNA      AACCTGCGGAAGGATCATTAACGGAGCCCGGAGGGCGAGGCCCGCGGCGGCGCCGCCGCC
NA19240_45S         AACCTGCGGAAGGATCATTAACGGAGCCCGGAGGGCGAGGCCCGCGGCGGCGCCGCCGCC
                    ************************************************************

reference_rDNA      GCCGCGCGCTTCCCTCCGCACACCCACCCCCCCACCGCGACGCGGCGCGTGCGCGGGCGG
NA19240_45S         GCCGCGCGCTTCCCTCCGCACACCCACCCCCCCACCGCGACGCGGCGCGTGCGCGGGCGG
                    ************************************************************

reference_rDNA      GGCCCGCGTGCCCGTTCGTTCGCTCGCTCGTTCGTTCGCCGCCCGGCCCCGCCGCCGCGA
NA19240_45S         GGCCCGCGTGCCCGTTCGTTCGCTCGCTCGTTCGTTCGCCGCCCGGCCCCGCCGCCGCGA
                    ************************************************************

reference_rDNA      GAGCCGAGAACTCGGGAGGGAGACGGGGGGGAGAGAGAGAGAGAGAGAGAGAGAGAGAGA
NA19240_45S         GAGCCGAGAACTCGGGAGGGAGACGGGGGGGAGAGAGAGAGAGAGAGAGAGAGAGAGAGA
                    ************************************************************

reference_rDNA      GAGAGAGAGAGAAAGAAGGGCGTGTCGTTGGTGTGCGCGTGTCGTGGGGCCGGCGGGCGG
NA19240_45S         GAGAGAGAGAGAAAGAAGGGCGTGTCGTTGGTGTGCGCGTGTCGTGGGGCCGGCGGGCGG
                    ************************************************************

reference_rDNA      CGGGGAGCGGTCCCCGGCCGCGGCCCCGACGACGTGGGTGTCGGCGGGCGCGGGGGCGGT
NA19240_45S         CGGGGAGCGGTCCCCGGCCGCGGCCCCGACGGCGTGGGTGTCGGCGGGCGCGGGGGCGGT
                    *******************************.****************************

reference_rDNA      TCTCGGCGGCGTCGCGGCGGGTCTGGGGGGGTCTCGGTGCCCTCCTCCCCGCCGGGGCCC
NA19240_45S         TCTCGGCGGCGTCGCGGCGGGTCTGGGG--GTCTCGGTGCCCTCCTCCCCGCCGGGGCCC
                    ****************************  ******************************

reference_rDNA      GTCGTCCGGCCCCGCCGCGCCGGCTCCCCGTCTTCGGGGCCGGCCGGATTCCCGTCGCCT
NA19240_45S         GTCGTCCGGCCCCGCCGCGCCGGCTCCCCGTCTTCGGGGCCGGCCGGATTCCCGTCGCCT
                    ************************************************************

reference_rDNA      CCGCCGCGCCGCTCCGCGCCGCCGGGCACGGCCCCGCTCGCTCTCCCCGGCCTTCCCGCT
NA19240_45S         CCGCCGCGCCGCTCCGCGCCGCCGGGCACGGCCCCGCTCGCTCTCCCCGGCCTTCCCGCT
                    ************************************************************

reference_rDNA      AGGGCGTCTCGAGGGTCGGGGGCCGGACGCCGGTCCCCTCCCCCGCCTCCTCGTCCGCCC
NA19240_45S         AGGGCGTCTCGAGGGTCGGGGGCCGGACGCCGGTCCCCTCCCCCGCCTCCTCGTCCGCCC
                    ************************************************************

reference_rDNA      CCCCGCCGTCCAGGTACCTAGCGCGTTCCGGCGCGGAGGTTTAAAGACCCCTTGGGGGGA
NA19240_45S         CCCCGCCGTCCAGGTACCTAGCGCGTTCCGGCGCGGAGGTTTAAAGACCCCTTGGGGGGA
                    ************************************************************

reference_rDNA      TCGCCCGTCCGCCCGTGGGTCGGGGGCGGTGGTGGGCCCGCGGGGGAGTCCCGTCGGGAG
NA19240_45S         TCGCCCGTCCGCCCGTGGGTCGGGGGCGGTGGTGGGCCCGCGGGGGAGTCCCGTCGGGAG
                    ************************************************************

reference_rDNA      GGGCCCGGCCCCTCCCGCGCCTCCACCGCGGACTCCGCTCCCCGGCCGGGGCCGCGCCGC
NA19240_45S         GGGCCCGGCCCCTCCCGCGCCTCCACCGCGGACTCCGCTCCCCGGCCGGGGCCGCGCCGC
                    ************************************************************

reference_rDNA      CGCCGCCGCCGCGGCGGCCGTCGGGTGGGGGCTTTACCCGGCGGCCGTCGCGCGCCTGCC
NA19240_45S         CGCCGCCGCCGCGGCGGCCGTCGGGTGGGGGCTTTACCCGGCGGCCGTCGCGCGCCTGCC
                    ************************************************************

reference_rDNA      GCGCGTGTGGCGTGCGCCCCGCGCCGTGGGGGCGGGAACCCCCGGGCGCCTGTGGGGTGG
NA19240_45S         GCGCGTGTGGCGTGCGCCCCGCGCCGTGGGGGCGGGAACCCCCGGGCGCCTGTGGGGTGG
                    ************************************************************

reference_rDNA      TGTCCGCGCTCGCCCCCGCGTGGGCGGCGCGCGCCTCCCCGTGGTGTGAAACCTTCCGAC
NA19240_45S         TGTCCGCGCTCGCCCCCGCGTGGGCGGCGCGCGCCTCCCCGTGGTGTGAAACCTTCCGAC
                    ************************************************************

reference_rDNA      CCCTCTCCGGAGTCCGGTCCCGTTTGCTGTCTCGTCTGGCCGGCCTGAGGCAACCCCCTC
NA19240_45S         CCCTCTCCGGAGTCCGGTCCCGTTTGCTGTCTCGTCTGGCCGGCCTGAGGCAACCCCCTC
                    ************************************************************

reference_rDNA      TCCTCTTGGGCGGGGGGGGCGGGGGGACGTGCCGCGCCAGGAAGGGCCTCCTCCCGGTGC
NA19240_45S         TCCTCTTGGGCGGGGGGGGCGGGGGGACGTGCCGCGCCAGGAAGGGCCTCCTCCCGGTGC
                    ************************************************************

reference_rDNA      GTCGTCGGGAGCGCCCTCGCCAAATCGACCTCGTACGACTCTTAGCGGTGGATCACTCGG
NA19240_45S         GTCGTCGGGAGCGCCCTCGCCAAATCGACCTCGTACGACTCTTAGCGGTGGATCACTCGG
                    ************************************************************

reference_rDNA      CTCGTGCGTCGATGAAGAACGCAGCTAGCTGCGAGAATTAATGTGAATTGCAGGACACAT
NA19240_45S         CTCGTGCGTCGATGAAGAACGCAGCTAGCTGCGAGAATTAATGTGAATTGCAGGACACAT
                    ************************************************************

reference_rDNA      TGATCATCGACACTTCGAACGCACTTGCGGCCCCGGGTTCCTCCCGGGGCTACGCCTGTC
NA19240_45S         TGATCATCGACACTTCGAACGCACTTGCGGCCCCGGGTTCCTCCCGGGGCTACGCCTGTC
                    ************************************************************

reference_rDNA      TGAGCGTCGCTTGCCGATCAATCG-CCCCGGGGGTGCCTCCGGGCTCCTCGGGGTGCGCG
NA19240_45S         TGAGCGTCGCTTGCCGATCAATCGCCCCCGGGGGTGCCTCCGGGCTCCTCGGGGTGCGCG
                    ************************ ***********************************

reference_rDNA      GCTGGGGGTTCCCTCGCAGGGCCCGCCGGGGGCCCTCCGTCCCCCTAAGCGCAGACCCGG
NA19240_45S         GCTGGGGGTTCCCTCGCAGGGCCCGCCGGGGGCCCTCCGTCCCCCTAAGCGCAGACCCGG
                    ************************************************************

reference_rDNA      CGGCGTCCGCCCTCCTCTTGCCGCCGCGCCCGCCCCTTCCCCCTCCCCCCGCGGGCCCTG
NA19240_45S         CGGCGTCCGCCCTCCTCTTGCCGCCGCGCCCGCCCCTTCCCCCTCCCCCCGCGGGCCCTG
                    ************************************************************

reference_rDNA      CGTGGTCACGCGTCGGGTGGCGGGGGGGAGAGGGGGGCGCGCCCGGCTGAGAGAGACGGG
NA19240_45S         CGTGGTCACGCGTCGGGTGGCGGGGGGGAGAGGGGGGCGCGCCCGGCTGAGAGAGACGGG
                    ************************************************************

reference_rDNA      GAGGGCGGCGCCGCCGCCG-----GAAGACGGAGAGGGAAAGAGAGAGCCGGCTCGGGCC
NA19240_45S         GAGGGCGGCGCCGCCGCCGCCCGCGAAGACGGAGAGGGAAAGAGAGAGCCGGCTCGGGCC
                    *******************     ************************************

reference_rDNA      GAGTTCCCGTGGCCGCCGCCTGCGGTCCGGGTTCCTCCCTCGGGGGGCTCCCTCGCGCCG
NA19240_45S         GAGTTCCCGTGGCCGCCGCCTGCGGTCCGGGTTCCTCCCTCGGGGGGCTCCCTCGCGCCG
                    ************************************************************

reference_rDNA      CGCGCGGCTCGGGGTTCGGGGTTCGTCGGCCCCGGCCGGGTGGAAGGTCCCGTGCCCGTC
NA19240_45S         CGCGCGGCTCGGGGTTCGGGGTTCGTCGGCCCCGGCCGGGTGGAAGGTCCCGTGCCCGTC
                    ************************************************************

reference_rDNA      GTCGTCGTCGTCGCGCGTCGTCGGCGGTGGGGGCGTGTTGCGTGCGGTGTGGTGGTGGGG
NA19240_45S         GTCGTCGTCGTCGCGCGTCGTCGGCGGTGGGGGCGTGTTGCGTGCGGTGTGGTGGTGGGG
                    ************************************************************

reference_rDNA      GAGGAGGAAGGCGGGTCCGGAAGGGGAAGGGTGCCGGCGGGGAGAGAGGGTCGGGGGAGC
NA19240_45S         GAGGAGGAAGGCGGGTCCGGAAGGGGAAGGGTGCCGGCGGGGAGAGAGGGTCGGGGGAGC
                    ************************************************************

reference_rDNA      GCGTCCCGGTCGCCGCGGTTCCGCCGCCCGCCCCCGGTGGCGGCCCGGCGTCCGGCCGAC
NA19240_45S         GCGTCCCGGTCGCCGCGGTTCCGCCGCCCGCCCCCGGTGGCGGCCCGGCGTCCGGCCGAC
                    ************************************************************

reference_rDNA      CGGCCGCTCCCCGCGCCCCTCCTCCTCCCCGCCGCCCCTCCTCCGAGGCCCCGCCCGTCC
NA19240_45S         CG-CCGCTCCCG-CGCCCCTCCTCCTCCCCGCCGCCCCTCCTCCGAGGCCCCGCCCGTCC
                    ** ********  ***********************************************

reference_rDNA      TCCTCGCCCTCCCCGCGCGTACGCGCGCGCGCCCGCCCGCCCGGCTCGCCTCGCGGCGCG
NA19240_45S         TCCTCGCCCTCCCCGCGCGTACGCGCGCGCGCCCGCCCGCCCGGCTCGCCTCGCGGCGCG
                    ************************************************************

reference_rDNA      TCGGCCGGGGCCGGGAGCCCGCCCCGCCGCCCGCCCG----------TGGCCGCGGCGCC
NA19240_45S         TCGGCCGGGGCCGGGAGCCCGCCCCGCGGCCCGCCCGGCCGCGCCCGTGGCCGCGGCGCC
                    *************************** *********          *************

reference_rDNA      GGGGTTCGCGTGTCCCCGGCGGCGACCCGCGGGACGCCGCGGTGTCGTCCGCCGTCGCGC
NA19240_45S         GGGGTTCGCGTGTCCCCGGCGGCGACCCGCGGGACGCCGCGGTGTCGTCCGCCGTCGCGC
                    ************************************************************

reference_rDNA      GCCCGCCTCCGGCTCGCGGCCGCGCCGCGCCGCGCCGGGGCCCCGTCCCGAGCTTCCGCG
NA19240_45S         GCCCGCCTCCGGCTCGCGGCCGCGCCGCGCCGCGCCGGGGCCCCGTCCCGAGCTTCCGCG
                    ************************************************************

reference_rDNA      TCGGGGCGGCGCGGCTCCGCCGCCGCGTCCTCGGACCCGTCCCCCCGACCTCCGCGGGGG
NA19240_45S         TCGGGGCGGGGCGGCTCCGCCGCCGCGTCCTCGGACCCGTCCCCCCGACCTCCGCGGGGG
                    ********* **************************************************

reference_rDNA      AGACGCGCCGGGGCGTGCGGCGCCCGTCCCGCCCCCGGCCCGTGCCCCTCCCTCCGGTCG
NA19240_45S         AGACGGGTCGGGGCGTGCGGCGCCCGTCCCGCCCCCGGCCCGTGCCCCTCCCTCCGGTCG
                    ***** * ****************************************************

reference_rDNA      TCCCGCTCCGGCGGGGCGGCGCGGGGGCGCCGTCGGCCGCGCGCTCTCTCTCCCGTCGCC
NA19240_45S         TCCCGCTCCGGCGGGGCGGCGCGGGGGTGCCGTCGGCCGCG-GCTCTCTCTCCCGTCGCC
                    *************************** ************* ******************

reference_rDNA      TCTCCCCCTCGCCGGGCCCGTCTCCCGACGGAGCGTCGGGCGGGCGGTCGGGCCGGCGCG
NA19240_45S         TCTCCCCCTCGCCGGGCCCGTCTCCCGACGGAGCGTCGGGCGGGCGGTCGGGCCGGCGCG
                    ************************************************************

reference_rDNA      ATTCCGTCCGTCCGTCCGCCGAGCGGCCCGTCCCCCTCCGAGACGCGACCTCAGATCAGA
NA19240_45S         ATTCCGTCCGTCCGTCCGCCGAGCGGCCCGTCCCCCTCCGAGACGCGACCTCAGATCAGA
                    ************************************************************

reference_rDNA      CGTGGCGACCCGCTGAATTTAAGCATATTAGTCAGCGGAGGAAAAGAAACTAACCAGGAT
NA19240_45S         CGTGGCGACCCGCTGAATTTAAGCATATTAGTCAGCGGAGGAGAAGAAACTAACCAGGAT
                    ******************************************.*****************

reference_rDNA      TCCCTCAGTAACGGCGAGTGAACAGGGAAGAGCCCAGCGCCGAATCCCCGCCCCGCG---
NA19240_45S         TCCCTCAGTAACGGCGAGTGAACAGGGAAGAGCCCAGCGCCGAATCCCCGCCCCGCGGCG
                    *********************************************************   

reference_rDNA      GGGCGCGGGACATGTGGCGTACGGAAGACCCGCTCCCCGGCGCCGCTCGTGGGGGGCCCA
NA19240_45S         GGGCGCGGGACATGTGGCGTACGGAAGACCCGCTCCCCGGCGCCGCTCGTGGGGGGCCCA
                    ************************************************************

reference_rDNA      AGTCCTTCTGATCGAGGCCCAGCCCGTGGACGGTGTGAGGCCGGTAGCGG---CCGGCGC
NA19240_45S         AGTCCTTCTGATCGAGGCCCAGCCCGTGGACGGTGTGAGGCCGGTAGCGGCCCCCGGCGC
                    **************************************************   *******

reference_rDNA      GCG---CCCGGGTCTTCCCGGAGTCGGGTTGCTTGGGAATGCAGCCCAAAGCGGGTGGTA
NA19240_45S         GCCGGGCCCGGGTCTTCCCGGAGTCGGGTTGCTTGGGAATGCAGCCCAAAGCGGGTGGTA
                    **    ******************************************************

reference_rDNA      AACTCCATCTAAGGCTAAATACCGGCACGAGACCGATAGTCAACAAGTACCGTAAGGGAA
NA19240_45S         AACTCCATCTAAGGCTAAATACCGGCACGAGACCGATAGTCAACAAGTACCGTAAGGGAA
                    ************************************************************

reference_rDNA      AGTTGAAAAGAACTTTGAAGAGAGAGTTCAAGAGGGCGTGAAACCGTTAAGAGGTAAACG
NA19240_45S         AGTTGAAAAGAACTTTGAAGAGAGAGTTCAAGAGGGCGTGAAACCGTTAAGAGGTAAACG
                    ************************************************************

reference_rDNA      GGTGGGGTCCGCGCAGTCCGCCCGGAGGATTCAACCCGGCGGCGGGTCCGGCCGTGTCGG
NA19240_45S         GGTGGGGTCCGCGCAGTCCGCCCGGAGGATTCAACCCGGCGGCGGGTCCGGCCGTGTCGG
                    ************************************************************

reference_rDNA      CGGCCCGGCGGATCTTTCCCGCCCCCCGTTCCTCCCGACCCCTCCACCCGCCCTCCCTTC
NA19240_45S         CGGCCCGGCGGATCTTTCCCGCCCCCCGTTCCTCCCGACCCCTCCACCCGCCCTCCCTTC
                    ************************************************************

reference_rDNA      CCCCGCCGCCCCTCCTCCTCCTCCCCGGAGGGGGCGGGCTCCGGCGGGTGCGGGGGTGGG
NA19240_45S         CCCCGCCGCCCCTCCTCCTCCTCCCCGGAGGGGGCGGGCTCCGGCGGGTGCGGGGGTGGG
                    ************************************************************

reference_rDNA      CGGGCGGGGCCGGGGGTGGGGTCGGCGGGGGACCGTCCCCCGACCGGCGACCGGCCGCCG
NA19240_45S         CGGGCGGGGCCGGGGGTGGGGTCGGCGGGGGACCGTCCCCCGACCGGCGACCGGCCGCCG
                    ************************************************************

reference_rDNA      CCGGGCGCATTTCCACCGCGGCGGTGCGCCGCGACCGGCTCCGGGACGGCTGGGAAGGCC
NA19240_45S         CCGGGCGCATTTCCACCGCGGCGGTGCGCCGCGACCGGCTCCGGGACGGCTGGGAAGGCC
                    ************************************************************

reference_rDNA      CGGCGGGGAAGGTGGCTCGGGGGGCCCCGTCCGTCCGTCCGTCCTCCTCCTCCCCCGTCT
NA19240_45S         CGGCGGGGAAGGTGGCTCGGGGGGCCCCGTCCGTCCGTCCGTCCTCCTCCTCCCCCGTCT
                    ************************************************************

reference_rDNA      CCGCCCCCCGGCCCCGCGTCCTCCCTCGGGAGGGCGCGCGGGTCGGGGCGGCGGCGGCGG
NA19240_45S         CCGCCCCCCGGCCCCGCGTCCTCCCTCGGGAGGGCGCGCGGGTCGGGGCGGCGGCGGCGG
                    ************************************************************

reference_rDNA      CGGCGGTGGCGGCGGCGGCGGGGGCGGCGGGACCGAAACCCCCCCCGAGTGTTACAGCCC
NA19240_45S         CGGCGGTGGCGGCGGCGGCGGCGGCGGCGGGACCGAAACCCCCCCCGAGTGTTACAGCCC
                    ********************* **************************************

reference_rDNA      CCCCGGCAGCAGCACTCGCCGAATCCCGGGGCCGAGGGAGCGAGACCCGTCGCCGCGCTC
NA19240_45S         CCCCGGCAGCAGCACTCGCCGAATCCCGGGGCCGAGGGAGCGAGACCCGTCGCCGCGCTC
                    ************************************************************

reference_rDNA      TCCCCCCTCCCGGCGCCCACCCCCGCGGGGAATCCCCCGCGAGGGGGGTCTCCCCCGCGG
NA19240_45S         TCCCCCCTCCCGGCGCCCACCCCCGCGGGGAATCCCCCGCGAGGGGGGTCTCCCCCGCGG
                    ************************************************************

reference_rDNA      GGGCGCGCCGGCGTCTCCTCGTGGGGGGGCCGGGCCACCCCTCCCACGGCGCGACCGCTC
NA19240_45S         GGGCGCGCCGGCGTCTCCTCGTGGGGGGGCCGGGCCACCCCTCCCACGGCGCGACCGCTC
                    ************************************************************

reference_rDNA      TCCCACCCCTCCTCCCCGCGCCCCCGCCCCGGCGACGGGGGGGGTGCCGCGCGCGGGTCG
NA19240_45S         TCCCACCCCTCCTCCCCGCGCCCCCGCCCCGGCGACGGGGGGGGTGCCGCGCGCGGGTCG
                    ************************************************************

reference_rDNA      GGGGGCGGGGCGGACTGTCCCCAGTGCGCCCCGGGCGGGTCGCGCCGTCGGGCCCGGGGG
NA19240_45S         GGGGGCGGGGCGGACTGTCCCCAGTGCGCCCCGGGCGGGTCGCGCCGTCGGGCCCGGGGG
                    ************************************************************

reference_rDNA      AGGTTCTCTCGGGGCCACGCGCGCGTCCCCCGAAGAGGGGGACGGCGGAGCGAGCGCACG
NA19240_45S         AGGTTCTCTCGGGGCCACGCGCGCGTCCCCCGAAGAGGGGGACGGCGGAGCGAGCGCACG
                    ************************************************************

reference_rDNA      GGGTCGGCGGCGACGTCGGCTACCCACCCGACCCGTCTTGAAACACGGACCAAGGAGTCT
NA19240_45S         GGGTCGGCGGCGACGTCGGCTACCCACCCGACCCGTCTTGAAACACGGACCAAGGAGTCT
                    ************************************************************

reference_rDNA      AACACGTGCGCGAGTCGGGGGCTCGCACGAAAGCCGCCGTGGCGCAATGAAGGTGAAGGC
NA19240_45S         AACACGTGCGCGAGTCGGGGGCTCGCACGAAAGCCGCCGTGGCGCAATGAAGGTGAAGGC
                    ************************************************************

reference_rDNA      CGGCGCGCTCGCCGGCCGAGGTGGGATCCCGAGGCCTCTCCAGTCCGCCGAGGGCGCACC
NA19240_45S         CGGCGCGCTCGCCGGCCGAGGTGGGATCCCGAGGCCTCTCCAGTCCGCCGAGGGCGCACC
                    ************************************************************

reference_rDNA      ACCGGCCCGTCTCGCCCGCCGCGCCGGGGAGGTGGAGCACGAGCGCACGTGTTAGGACCC
NA19240_45S         ACCGGCCCGTCTCGCCCGCCGCGCCGGGGAGGTGGAGCACGAGCGCACGTGTTAGGACCC
                    ************************************************************

reference_rDNA      GAAAGATGGTGAACTATGCCTGGGCAGGGCGAAGCCAGAGGAAACTCTGGTGGAGGTCCG
NA19240_45S         GAAAGATGGTGAACTATGCCTGGGCAGGGCGAAGCCAGAGGAAACTCTGGTGGAGGTCCG
                    ************************************************************

reference_rDNA      TAGCGGTCCTGACGTGCAAATCGGTCGTCCGACCTGGGTATAGGGGCGAAAGACTAATCG
NA19240_45S         TAGCGGTCCTGACGTGCAAATCGGTCGTCCGACCTGGGTATAGGGGCGAAAGACTAATCG
                    ************************************************************

reference_rDNA      AACCATCTAGTAGCTGGTTCCCTCCGAAGTTTCCCTCAGGATAGCTGGCGCTCTCGCAGA
NA19240_45S         AACCATCTAGTAGCTGGTTCCCTCCGAAGTTTCCCTCAGGATAGCTGGCGCTCTCGCAGA
                    ************************************************************

reference_rDNA      CCCGACGCACCCCCGCCACGCAGTTTTATCCGGTAAAGCGAATGATTAGAGGTCTTGGGG
NA19240_45S         CCCGACGCACCCCCGCCACGCAGTTTTATCCGGTAAAGCGAATGATTAGAGGTCTTGGGG
                    ************************************************************

reference_rDNA      CCGAAACGATCTCAACCTATTCTCAAACTTTAAATGGGTAAGAAGCCCGGCTCGCTGGCG
NA19240_45S         CCGAAACGATCTCAACCTATTCTCAAACTTTAAATGGGTAAGAAGCCCGGCTCGCTGGCG
                    ************************************************************

reference_rDNA      TGGAGCCGGGCGTGGAATGCGAGTGCCTAGTGGGCCACTTTTGGTAAGCAGAACTGGCGC
NA19240_45S         TGGAGCCGGGCGTGGAATGCGAGTGCCTAGTGGGCCACTTTTGGTAAGCAGAACTGGCGC
                    ************************************************************

reference_rDNA      TGCGGGATGAACCGAACGCCGGGTTAAGGCGCCCGATGCCGACGCTCATCAGACCCCAGA
NA19240_45S         TGCGGGATGAACCGAACGCCGGGTTAAGGCGCCCGATGCCGACGCTCATCAGACCCCAGA
                    ************************************************************

reference_rDNA      AAAGGTGTTGGTTGATATAGACAGCAGGACGGTGGCCATGGAAGTCGGAATCCGCTAAGG
NA19240_45S         AAAGGTGTTGGTTGATATAGACAGCAGGACGGTGGCCATGGAAGTCGGAATCCGCTAAGG
                    ************************************************************

reference_rDNA      AGTGTGTAACAACTCACCTGCCGAATCAACTAGCCCTGAAAATGGATGGCGCTGGAGCGT
NA19240_45S         AGTGTGTAACAACTCACCTGCCGAATCAACTAGCCCTGAAAATGGATGGCGCTGGAGCGT
                    ************************************************************

reference_rDNA      CGGGCCCATACCCGGCCGTCGCCGGCAGTCGAGAGTGGACGGGAGCGGCGGGGGCGGCGC
NA19240_45S         CGGGCCCATACCCGGCCGTCGCCGGCAGTCGAGAGTGGACGGGAGCGGCGGGGGCGGCGC
                    ************************************************************

reference_rDNA      GCGCGCGCGCGCGTGTGGTGTGCGTCGGAGGGCGGCGGCGGCGGCGGCGGCGGGGGTGTG
NA19240_45S         GCGCGCGCGCGCGTGTGGTGTGCGTCGGAGGGCGGCGGCGGCGGCGGCGGCGGGGGTGTG
                    ************************************************************

reference_rDNA      GGGTCCTTCCCCCGCCCCCCCCCCCACGCCTCCTCCCCTCCTCCCGCCCACGCCCCGCTC
NA19240_45S         GGGTCCTTCCCCCGCCCCCCCCCCCACGCCTCCTCCCCTCCTCCCGCCCACGCCCCGCTC
                    ************************************************************

reference_rDNA      CCCGCCCCCGGAGCCCCGCGGACGCTACGCCGCGACGAGTAGGAGGGCCGCTGCGGTGAG
NA19240_45S         CCCGCCCCCGGAGCCCCGCGGACGCTACGCCGCGACGAGTAGGAGGGCCGCTGCGGTGAG
                    ************************************************************

reference_rDNA      CCTTGAAGCCTAGGGCGCGGGCCCGGGTGGAGCCGCCGCAGGTGCAGATCTTGGTGGTAG
NA19240_45S         CCTTGAAGCCTAGGGCGCGGGCCCGGGTGGAGCCGCCGCAGGTGCAGATCTTGGTGGTAG
                    ************************************************************

reference_rDNA      TAGCAAATATTCAAACGAGAACTTTGAAGGCCGAAGTGGAGAAGGGTTCCATGTGAACAG
NA19240_45S         TAGCAAATATTCAAACGAGAACTTTGAAGGCCGAAGTGGAGAAGGGTTCCATGTGAACAG
                    ************************************************************

reference_rDNA      CAGTTGAACATGGGTCAGTCGGTCCTGAGAGATGGGCGAGCGCCGTTCCGAAGGGACGGG
NA19240_45S         CAGTTGAACATGGGTCAGTCGGTCCTGAGAGATGGGCGAGCGCCGTTCCGAAGGGACGGG
                    ************************************************************

reference_rDNA      CGATGGCCTCCGTTGCCCTCGGCCGATCGAAAGGGAGTCGGGTTCAGATCCCCGAATCCG
NA19240_45S         CGATGGCCTCCGTTGCCCTCGGCCGATCGAAAGGGAGTCGGGTTCAGATCCCCGAATCCG
                    ************************************************************

reference_rDNA      GAGTGGCGGAGATGGGCGCCGCGAGGCGTCCAGTGCGGTAACGCGACCGATCCCGGAGAA
NA19240_45S         GAGTGGCGGAGATGGGCGCCGCGAGGCGTCCAGTGCGGTAACGCGACCGATCCCGGAGAA
                    ************************************************************

reference_rDNA      GCCGGCGGGAGCCCCGGGGAGAGTTCTCTTTTCTTTGTGAAGGGCAGGGCGCCCTGGAAT
NA19240_45S         GCCGGCGGGAGCCCCGGGGAGAGTTCTCTTTTCTTTGTGAAGGGCAGGGCGCCCTGGAAT
                    ************************************************************

reference_rDNA      GGGTTCGCCCCGAGAGAGGGGCCCGTGCCTTGGAAAGCGTCGCGGTTCCGGCGGCGTCCG
NA19240_45S         GGGTTCGCCCCGAGAGAGGGGCCCGTGCCTTGGAAAGCGTCGCGGTTCCGGCGGCGTCCG
                    ************************************************************

reference_rDNA      GTGAGCTCTCGCTGGCCCTTGAAAATCCGGGGGAGAGGGTGTAAATCTCGCGCCGGGCCG
NA19240_45S         GTGAGCTCTCGCTGGCCCTTGAAAATCCGGGGGAGAGGGTGTAAATCTCGCGCCGGGCCG
                    ************************************************************

reference_rDNA      TACCCATATCCGCAGCAGGTCTCCAAGGTGAACAGCCTCTGGCATGTTGGAACAATGTAG
NA19240_45S         TACCCATATCCGCAGCAGGTCTCCAAGGTGAACAGCCTCTGGCATGTTGGAACAATGTAG
                    ************************************************************

reference_rDNA      GTAAGGGAAGTCGGCAAGCCGGATCCGTAACTTCGGGATAAGGATTGGCTCTAAGGGCTG
NA19240_45S         GTAAGGGAAGTCGGCAAGCCGGATCCGTAACTTCGGGATAAGGATTGGCTCTAAGGGCTG
                    ************************************************************

reference_rDNA      GGTCGGTCGGGCTGGGGCGCGAAGCGGGGCTGGGCGCGCGCCGCGGCTGGACGAGGCGCG
NA19240_45S         GGTCGGTCGGGCTGGGGCGCGAAGCGGGGCTGGGCGCGCGCCGCGGCTGGACGAGGCGCC
                    *********************************************************** 

reference_rDNA      --CGCCCCCCCCACGCCCGGGGCACCCCCCTCGCGGCCCTCCCCCGCCCCACCCG-CGCG
NA19240_45S         GCCGCCCCCCCCACGCCCGGGGCACCCCCCTCGCGGCCCTCCCCCGCCCCACCCCGCGCG
                      ****************************************************  ****

reference_rDNA      CGCCGCTCGCTCCCTCCCCACCCCGCGCCCTCTCTCTCTCTCTCTCCCCCGCTCCCCGTC
NA19240_45S         CGCCGCTCGCTCCCTCCCCACCCCGCGCCCTCTCTCTCTCTCTCTCCCCCGCTCCCCGTC
                    ************************************************************

reference_rDNA      CTCCCCCCTCCCCGGGGGAGCGCCGCGTGGGGGCG-CGGCGGGGGGAGAAGGGTCGGGGC
NA19240_45S         CTCCCCCCTCCCCGGGGGAGCGCCGCGTGGGGGCGGCGGCGGGGGGAGAAGGGTCGGGGC
                    *********************************** ************************

reference_rDNA      GGCAGGGGCCGCGCGGCGG-CCGCCGGGG----CGGCCGGCGGGGGCAGG--TCCCCGCG
NA19240_45S         GGCAGGGGCCG-GCGGCGGCCCGCCGCGGGGCCCCGGCGGCGGGGGCACGGTCCCCCGCG
                    *********** ******* ****** **    * * *********** *   *******

reference_rDNA      AGGGGGGCCCCGGGGACCCGGGGGGCCGGCGGCGGCGCGGACTCTGGACGCGAGCCGGGC
NA19240_45S         AGGGGGG-CCCGGGCACCCGGGGGGCCGGCGGCGGCGGCGACTCTGGACGCGAGCCGGGC
                    ******* ****** **********************  *********************

reference_rDNA      CCTTCCCGTGGATCGCCCCAGCTGCGGCGGGCGTCGCGGCCGCCCCCGGGGAGCCCGGCG
NA19240_45S         CCTTCCCGTGGATCGCCCCAGCTGCGGCGGGCGTCGCGGCCGCCCCCGGGGAGCCCGGCG
                    ************************************************************

reference_rDNA      GCGG-CGCGGCGCGCCCCCCACCCCCACCCCACGTCTCGGTCGCGCGCGCGTCCGCTGGG
NA19240_45S         GCGGGCGCG-CGCGCCCCCCACCCCCACCCCACGTCTCG-TCGCGCGCGCGTCCGCTGGG
                    **** **** ***************************** ********************

reference_rDNA      GGCGGG-AGCGGTCGGGCGGCGGCGGTCGGCGGGCGGCGGGGCGGGGCGGTTCGTCCCCC
NA19240_45S         GGCGGGGAGCGGTCGGGCGGCGGCGGTCGGCGGGCGGCGGGGCGGGGCGGTTCGTCCCCC
                    ****** *****************************************************

reference_rDNA      CGCCCTACCCCCCCGGCCCCGTCCGCCCCCCGTTCCCCCCTCCTCCTCGGCGCGCGGCGG
NA19240_45S         CGCCCTACCCCCCCGGCCCCGTCCGCCCCCCGTTCCCCCCTCCTCCTCGGCGCGCGGCGG
                    ************************************************************

reference_rDNA      CGGCGGCGGCAGGCGGCGGAGGGGCCGCGGGCCGGTCCCCCCCGCCGGGTCCGCCCCCGG
NA19240_45S         CGGCGGCGGCAGGCGGCGGAGGGGCCGCGGGCCGGTCCCCCCCGCCGGGTCCGCCCCCGG
                    ************************************************************

reference_rDNA      GGCCGCGGTTCCGCGCGCG-CCTCGCCTCGGCCGGCGCCTAGCAGCCGACTTAGAACTGG
NA19240_45S         GGCCGCGGTTCCGCGCGCGGCCTCGCCTCGGCCGGCGCCTAGCAGCCGACTTAGAACTGG
                    ******************* ****************************************

reference_rDNA      TGCGGACCAGGGGAATCCGACTGTTTAATTAAAACAAAGCATCGCGAAGGCCCGCGGCGG
NA19240_45S         TGCGGACCAGGGGAATCCGACTGTTTAATTAAAACAAAGCATCGCGAAGGCCCGCGGCGG
                    ************************************************************

reference_rDNA      GTGTTGACGCGATGTGATTTCTGCCCAGTGCTCTGAATGTCAAAGTGAAGAAATTCAATG
NA19240_45S         GTGTTGACGCGATGTGATTTCTGCCCAGTGCTCTGAATGTCAAAGTGAAGAAATTCAATG
                    ************************************************************

reference_rDNA      AAGCGCGGGTAAACGGCGGGAGTAACTATGACTCTCTTAAGGTAGCCAAATGCCTCGTCA
NA19240_45S         AAGCGCGGGTAAACGGCGGGAGTAACTATGACTCTCTTAAGGTAGCCAAATGCCTCGTCA
                    ************************************************************

reference_rDNA      TCTAATTAGTGACGCGCATGAATGGATGAACGAGATTCCCACTGTCCCTACCTACTATCC
NA19240_45S         TCTAATTAGTGACGCGCATGAATGGATGAACGAGATTCCCACTGTCCCTACCTACTATCC
                    ************************************************************

reference_rDNA      AGCGAAACCACAGCCAAGGGAACGGGCTTGGCGGAATCAGCGGGGAAAGAAGACCCTGTT
NA19240_45S         AGCGAAACCACAGCCAAGGGAACGGGCTTGGCGGAATCAGCGGGGAAAGAAGACCCTGTT
                    ************************************************************

reference_rDNA      GAGCTTGACTCTAGTCTGGCACGGTGAAGAGACATGAGAGGTGTAGAATAAGTGGGAGGC
NA19240_45S         GAGCTTGACTCTAGTCTGGCACGGTGAAGAGACATGAGAGGTGTAGAATAAGTGGGAGGC
                    ************************************************************

reference_rDNA      CCCCGGCGCCCCCCCGGTGTCCCCGCGAGGGGCCCGGGGCGGGGTCCG-CGGCCCTGCGG
NA19240_45S         CCCCGGCGCCCCCCCGGTGTCCCCGCGAGGGGCCCGGGGCGGGGTCCGCCGGCCCTGCGG
                    ************************************************ ***********

reference_rDNA      GCCG-CCGGTGAAATACCACTACTCTGATCGTTTTTTCACTGACCCGGTGAGGCGGGGGG
NA19240_45S         GCCGCCCGGTGAAATACCACTACTCTGATCGTTTTTTCACTGACCCGGTGAGGCGGGGGG
                    **** *******************************************************

reference_rDNA      GCGAG-CCCGAGGGGCTCTCGCTTCTGGCGCCAAGCGCCCG-------CCCGGCCGGGCG
NA19240_45S         GCGAGCCCCGAGGGGCTCTCGCTTCTGGCGCCAAGCGCCCGGCCGCGCGCCGGCCGGGCG
                    ***** ***********************************        ***********

reference_rDNA      CGACCCGCTCCGGGGACAGTGCCAGGTGGGGAGTTTGACTGGGGCGGTACACCTGTCAAA
NA19240_45S         CGACCCGCTCCGGGGACAGTGCCAGGTGGGGAGTTTGACTGGGGCGGTACACCTGTCAAA
                    ************************************************************

reference_rDNA      CGGTAACGCAGGTGTCCTAAGGCGAGCTCAGGGAGGACAGAAACCTCCCGTGGAGCAGAA
NA19240_45S         CGGTAACGCAGGTGTCCTAAGGCGAGCTCAGGGAGGACAGAAACCTCCCGTGGAGCAGAA
                    ************************************************************

reference_rDNA      GGGCAAAAGCTCGCTTGATCTTGATTTTCAGTACGAATACAGACCGTGAAAGCGGGGCCT
NA19240_45S         GGGCAAAAGCTCGCTTGATCTTGATTTTCAGTACGAATACAGACCGTGAAAGCGGGGCCT
                    ************************************************************

reference_rDNA      CACGATCCTTCTGACCTTTTGGGTTTTAAGCAGGAGGTGTCAGAAAAGTTACCACAGGGA
NA19240_45S         CACGATCCTTCTGACCTTTTGGGTTTTAAGCAGGAGGTGTCAGAAAAGTTACCACAGGGA
                    ************************************************************

reference_rDNA      TAACTGGCTTGTGGCGGCCAAGCGTTCATAGCGACGTCGCTTTTTGATCCTTCGATGTCG
NA19240_45S         TAACTGGCTTGTGGCGGCCAAGCGTTCATAGCGACGTCGCTTTTTGATCCTTCGATGTCG
                    ************************************************************

reference_rDNA      GCTCTTCCTATCATTGTGAAGCAGAATTCGCCAAGCGTTGGATTGTTCACCCACTAATAG
NA19240_45S         GCTCTTCCTATCATTGTGAAGCAGAATTCACCAAGCGTTGGATTGTTCACCCACTAATAG
                    *****************************.******************************

reference_rDNA      GGAACGTGAGCTGGGTTTAGACCGTCGTGAGACAGGTTAGTTTTACCCTACTGATGATGT
NA19240_45S         GGAACGTGAGCTGGGTTTAGACCGTCGTGAGACAGGTTAGTTTTACCCTACTGATGATGT
                    ************************************************************

reference_rDNA      GTTGTTGCCATGGTAATCCTGCTCAGTACGAGAGGAACCGCAGGTTCAGACATTTGGTGT
NA19240_45S         GTTGTTGCCATGGTAATCCTGCTCAGTACGAGAGGAACCGCAGGTTCAGACATTTGGTGT
                    ************************************************************

reference_rDNA      ATGTGCTTGGCTGAGGAGCCAATGGGGCGAAGCTACCATCTGTGGGATTATGACTGAACG
NA19240_45S         ATGTGCTTGGCTGAGGAGCCAATGGGGCGAAGCTACCATCTGTGGGATTATGACTGAACG
                    ************************************************************

reference_rDNA      CCTCTAAGTCAGAATCCCGCCCAGGCG-AACGATACGGCAGCGCCGCGGAGCCTCGGTTG
NA19240_45S         CCTCTAAGTCAGAATCCCGCCCAGGCGGAACGATACGGCAGCGCCGCGGAGCCTCGGTTG
                    *************************** ********************************

reference_rDNA      GCCTCGGATAGCCGGTCCCCCGCCTGTCCCCGCCGGCGGGCCG-CCCCCCCCTCCACGCG
NA19240_45S         GCCTCGGATAGCCGGTCCCCCGCCTGTCCCCGCCGGCGGGCCGCCCCCCCCCTCCACGCG
                    ******************************************* ****************

reference_rDNA      CCCCGC---CGCGGGAGGGCGCGTGCCCCGCCGCGCGCCGGGACCGGGGTCCGGTGCGGA
NA19240_45S         CCCCGCGCGCGCGGGAGGGCGCGTGCCCCGCCGCGCGCCGGGACCGGGGTCCGGTGCGGA
                    ******   ***************************************************

reference_rDNA      GTGCCCTTCGTCCTGGGAAACGGGGCGCGGCCGGAAAGGCGGCCGCCCCCTCGCCCGTCA
NA19240_45S         GTGCCCTTCGTCCTGGGAAACGGGGCGCGGCCGGAAAGGCGGCCGCCCCCTCGCCCGTCA
                    ************************************************************

reference_rDNA      CGCACCGCACGTTCGTGGGGAACCTGGCGCTAAACCATTCGTAGACGACCTGCTTCTGGG
NA19240_45S         CGCACCGCACGTTCGTGGGGAACCTGGCGCTAAACCATTCGTAGACGACCTGCTTCTGGG
                    ************************************************************

reference_rDNA      TCGGGGTTTCGTACGTAGCAGAGCAGCTCCCTCGCTGCGATCTATTGAAAGTCAGCCCTC
NA19240_45S         TCGGGGTTTCGTACGTAGCAGAGCAGCTCCCTCGCTGCGATCTATTGAAAGTCAGCCCTC
                    ************************************************************

reference_rDNA      GACACAAGGGTTTGTCCGCGCGCGCGTGCGTGCGGGGGGCCCGGCGGG-CGTGCGCGTTC
NA19240_45S         GACACAAGGGTTTGTCCGCGCGCGCGTGCGTGCGGGGGGCCCGGCGGGGCGTGCGCGCTC
                    ************************************************ ******** **

reference_rDNA      GGCGCCGTCCGTCCTTCCGTTCGTCTTCCTCCCTCCCGGCCTCTCCCGCCGACCGCGG-C
NA19240_45S         GGCGCCGTCCGTCCTTCCGTTCGTCTTCCTCCCTCCCGGCCTCTCCCGCCGACCGCGGGC
                    ********************************************************** *

reference_rDNA      GTGGTGGTGGGG-TGGGGGGGAGGGCGCGCGACCCCGGTCGGC-CGCCCCGCTTCTTCGG
NA19240_45S         GTGGTGGTGGGGGTGGGGGGGAGGGCGCGCGACCCCGGTCGGCGCGCCCCGCTTCTTCGG
                    ************ ****************************** ****************

reference_rDNA      TTCCCGCCTCCTCCCCGTTCACG-CCGGGGCGGCTCGTCCGCTCCGGGCCGGGACGGGGT
NA19240_45S         TTCCCGCCTCCTCCCCGTTCACCGCCGGGGCGGCTCGTCCGCTCCGGGCCGGGACGGGGT
                    **********************  ************************************

reference_rDNA      CCGGGGAGCGTGGTTTGGGAGCCGCGGAGGCG-CCGCGCCGAGCCGGGCCCCGTGGCCCG
NA19240_45S         CCGGGGAGCGTGGTTTGGGAGCCGCGGAGGCGGCCGCGCCGAGCCGGG-CCCGTGGCCCG
                    ******************************** *************** ***********

reference_rDNA      CCGGTCCCCGTCCCGGGGGTTGGCCGCGCGG--CGCGGTGGGG-GGCCACCCGGGGTCCC
NA19240_45S         CCGGTCCCCGTCCCGGGGGTTGGCCGCGCGGGCCCCGGTGGGGCGGCCACCCGGGGTCCC
                    *******************************  * ******** ****************

reference_rDNA      GGCCCTCGCGCGTCCTTCCTCCTCGCTCCTCCGCACGGGTCGACCGACGAACCGCGGGTG
NA19240_45S         GGCCCTCGCGCGTCCTTCCTCCTCGCTCCTCCGCACGGGTCGACCAGCAGACCGCGGGTG
                    *********************************************..*..**********

reference_rDNA      GCGGGCGGCGGGCGGCGAG-CCCCACGGG-CGTCCCCGCACCCGGCCGACCTCCGCTCGC
NA19240_45S         GCGGGCGGCGGGCGGCGAGGCCCCACGGGGCGTCCCCGCACCCGGCCGACCTCCGCTCGC
                    ******************* ********* ******************************

reference_rDNA      GACCTCTCCTCGGTCGGGCCTCCGGGGTCGACCGCCTG-CGCCCGCGGGCGTGAGACTCA
NA19240_45S         GACCTCTCCTCGGTCGGGCCTCCGGGGTCGACCGCCTGCCGCCCGCGGGCGTGAGACTCA
                    ************************************** *********************

reference_rDNA      G-CGGCGTCTCGCCGTGTCC
NA19240_45S         GCCGGCGTCTCGCCGTGTCC
                    * ******************
                    ```

### Alternative Haplotype 4870
Analysis of Haplotype 4870 from NA19240; what can it be?

![Alternative Haplotype 18s 4904](../figure/20170130_18s.altHaplo4910.png)

An example locus where many variants cluster together and I'm calling an alternative haplotype of the rDNA repeat. As mentioned later, these may represent variant rRNA which have multiple differences from the reference or it may represent pseudo-rDNA which has mutated over evolutionary time.

RNA studies (Adaconis) will be the best way to anwer this once alternative haplotypes of rDNA are created to align against.

#### Protein Contacts
![Alternative Haplotype Protein Contacts](../figure/20170126_altHap_protContacts.png)

#### Base Pair Contacts
![Alternative Haplotype Base Pairs](../figure/20170126_altHap_basepairs.png)


#### Blast Search - alt4870

```
>18s.refHap.4870
CGGCGATGCGGCGGCGTTATTCCCATGACCCGCCGGGCAGCTTCCGGGAA
ACCAAAGTCTTTGGGTTCCGGGGGGAGTATGGTTGCAAAGCTGAAACTTA
AAGGAATTGACGGAAGGGCACCACCAGGAGTGGAGCCTGCGGCTTAATTT
GACTCAACACGGGAAACCTCACCCGGCCCGGACACGGACAGGATTGACAG
ATTGATAGCTCTTTCTCGATTCCGTGGGTGGTGGTGCATGGCCGTTCTTA

>18s.altHap.4870
CGGCGATGCGGCGGCGTTATTCCCATGACCCGCTGGGCAGCTTCCAAGA
AACCAAAGTCTTTGggtttttaGTTCCGGGGGGAGTATGGTTGCAAAGC
TGAAACTTAAAGGAATTGGTGGAAGGGACCAGCAGGAGTGGCGCCTATG
GCTTAATTTGACTCAACATGGGAAAAATCACCCGGCCGGACATGGACAG
GATTGACAG

>Pan_troglodytes/1118-1179
GACCCGCUGGGCAGCUUCCAAGAAACCAAAGUCUUUGCGUUUUUAGGUUC
CGGGGGGAGUAU
```

| Query | # | reference | accession | match |
|-------|---|-----------|-----------|-------|
|18s.altHap.4870,gi|28201761|gb|AC129664.7|,18s.altHap.4870,AC129664.7,98.537,205,2,1,2,205,103766,103562,4.29e-96,361|
|18s.altHap.4870,gi|27901956|gb|AC127385.6|,18s.altHap.4870,AC127385.6,98.049,205,3,1,2,205,18218,18422,1.99e-94,355|
|18s.altHap.4870,gi|39978746|emb|AL162451.16|,18s.altHap.4870,AL162451.16,98.049,205,3,1,2,205,55169,54965,1.99e-94,355|
|18s.altHap.4870,gi|164414496|gb|AC200706.3|,18s.altHap.4870,AC200706.3,95.545,202,7,2,5,205,74128,73928,2.02e-84,322|
|18s.altHap.4870,gi|440500693|gb|JX395930.1|,18s.altHap.4870,JX395930.1,98.876,178,1,1,2,178,421,598,9.41e-83,316|
|18s.altHap.4870,gi|118636074|emb|CT978678.7|,18s.altHap.4870,CT978678.7,94.231,208,5,6,2,205,19265,19469,4.38e-81,311|
|18s.altHap.4870,gi|894216365|gb|AC270088.1|,18s.altHap.4870,AC270088.1,94.472,199,8,3,10,205,4641,4443,7.33e-79,303|
|18s.altHap.4870,gi|440500715|gb|JX395952.1|,18s.altHap.4870,JX395952.1,94.472,199,8,3,10,205,440,638,7.33e-79,303|
|18s.altHap.4870,gi|31745199|gb|AC136932.4|,18s.altHap.4870,AC136932.4,94.472,199,8,3,10,205,59551,59353,7.33e-79,303|
|18s.altHap.4870,gi|18093149|gb|AC018688.9|,18s.altHap.4870,AC018688.9,92.157,204,11,4,2,202,139391,139592,9.55e-73,283|
|18s.altHap.4870,gi|109689924|gb|AC183378.2|,18s.altHap.4870,AC183378.2,92.462,199,12,3,10,205,4298,4100,3.43e-72,281|
|18s.altHap.4870,gi|28460740|gb|AC137488.2|,18s.altHap.4870,AC137488.2,90.821,207,16,3,2,205,40845,40639,5.75e-70,274|
|18s.altHap.4870,gi|300794814|ref|NG_002801.3|,18s.altHap.4870,NG_002801.3,90.338,207,17,3,2,205,1176,1382,2.67e-68,268|
|18s.altHap.4870,gi|113869733|gb|AC150907.6|,18s.altHap.4870,AC150907.6,90.686,204,16,3,5,205,59072,58869,2.67e-68,268|
|18s.altHap.4870,gi|48958804|gb|AC147656.3|,18s.altHap.4870,AC147656.3,90.686,204,16,3,5,205,26432,26229,2.67e-68,268|
|18s.altHap.4870,gi|45120404|gb|AC148473.1|,18s.altHap.4870,AC148473.1,90.686,204,16,3,5,205,26728,26525,2.67e-68,268|
|18s.altHap.4870,gi|63028375|gb|AC146250.5|,18s.altHap.4870,AC146250.5,90.338,207,17,3,2,205,167567,167773,2.67e-68,268|
|18s.altHap.4870,gi|61506889|dbj|BS000664.2|,18s.altHap.4870,BS000664.2,90.338,207,17,3,2,205,35081,35287,2.67e-68,268|
|18s.altHap.4870,gi|11024927|gb|AC010970.3|AC010970,18s.altHap.4870,AC010970.3,90.338,207,17,3,2,205,23157,23363,2.67e-68,268|
|18s.altHap.4870,gi|24371525|gb|AC097532.3|,18s.altHap.4870,AC097532.3,90.338,207,15,4,2,205,60274,60070,9.62e-68,267|
|18s.altHap.4870,gi|45238689|gb|AC147660.2|,18s.altHap.4870,AC147660.2,89.855,207,18,3,2,205,149432,149638,1.24e-66,263|
|18s.altHap.4870,gi|116734383|gb|EF015892.1|,18s.altHap.4870,EF015892.1,89.806,206,12,3,2,205,115,313,2.08e-64,255|
|18s.altHap.4870,gi|969813047|pdb|5FLX|1,18s.altHap.4870,5FLX_1,89.372,207,13,3,1,205,1095,1294,2.69e-63,252|
|18s.altHap.4870,gi|887492877|pdb|5A2Q|2,18s.altHap.4870,5A2Q_2,89.372,207,13,3,1,205,1095,1294,2.69e-63,252|
|18s.altHap.4870,gi|665764225|pdb|4CXC|1;gi753536039|pdb|4D5L|1;gi764090858|pdb|4D61|1,18s.altHap.4870,4CXC_1,89.372,207,13,3,1,205,1095,1294,2.69e-63,252|
|18s.altHap.4870,gi|531997082|gb|KC876030.1|,18s.altHap.4870,KC876030.1,89.372,207,13,3,1,205,21755,21556,2.69e-63,252|
|18s.altHap.4870,gi|531997081|gb|KC876029.1|,18s.altHap.4870,KC876029.1,89.372,207,13,3,1,205,28845,28646,2.69e-63,252|
|18s.altHap.4870,gi|531997080|gb|KC876028.1|,18s.altHap.4870,KC876028.1,89.372,207,13,3,1,205,37021,36822,2.69e-63,252|
|18s.altHap.4870,gi|531997079|gb|KC876027.1|,18s.altHap.4870,KC876027.1,89.372,207,13,3,1,205,33963,33764,2.69e-63,252|
|18s.altHap.4870,gi|374429547|ref|NR_046235.1|,18s.altHap.4870,NR_046235.1,89.372,207,13,3,1,205,4749,4948,2.69e-63,252|
|18s.altHap.4870,gi|373861012|gb|HQ865942.1|,18s.altHap.4870,HQ865942.1,89.372,207,13,3,1,205,484,683,2.69e-63,252|
|18s.altHap.4870,gi|291497478|gb|GU825109.1|,18s.altHap.4870,GU825109.1,89.372,207,13,3,1,205,461,660,2.69e-63,252|
|18s.altHap.4870,gi|291496623|gb|GU824254.1|,18s.altHap.4870,GU824254.1,89.372,207,13,3,1,205,468,667,2.69e-63,252|
|18s.altHap.4870,gi|307691242|ref|NR_036642.1|,18s.altHap.4870,NR_036642.1,89.372,207,13,3,1,205,1095,1294,2.69e-63,252|
|18s.altHap.4870,gi|297747482|gb|AC231275.2|,18s.altHap.4870,AC231275.2,89.372,207,13,3,1,205,31737,31936,2.69e-63,252|
|18s.altHap.4870,gi|296434039|emb|FP236383.15|,18s.altHap.4870,FP236383.15,89.372,207,13,3,1,205,82789,82988,2.69e-63,252|
|18s.altHap.4870,gi|296434039|emb|FP236383.15|,18s.altHap.4870,FP236383.15,89.372,207,13,3,1,205,126999,127198,2.69e-63,252|
|18s.altHap.4870,gi|225637497|ref|NR_003286.2|,18s.altHap.4870,NR_003286.2,89.372,207,13,3,1,205,1095,1294,2.69e-63,252|
|18s.altHap.4870,gi|193507772|gb|EU811267.1|,18s.altHap.4870,EU811267.1,89.372,207,13,3,1,205,246,445,2.69e-63,252|
|18s.altHap.4870,gi|187609247|pdb|2ZKQ|A,18s.altHap.4870,2ZKQ_A,89.372,207,13,3,1,205,820,1019,2.69e-63,252|
|18s.altHap.4870,gi|157677450|emb|CT476837.18|,18s.altHap.4870,CT476837.18,89.372,207,13,3,1,205,98799,98998,2.69e-63,252|
|18s.altHap.4870,gi|151415227|emb|AL353644.34|,18s.altHap.4870,AL353644.34,89.372,207,13,3,1,205,127025,127224,2.69e-63,252|
|18s.altHap.4870,gi|151415227|emb|AL353644.34|,18s.altHap.4870,AL353644.34,89.372,207,13,3,1,205,172106,172305,2.69e-63,252|
|18s.altHap.4870,gi|5714635|gb|AF159295.1|AF159295,18s.altHap.4870,AF159295.1,89.372,207,13,3,1,205,784,585,2.69e-63,252|
|18s.altHap.4870,gi|86475748|emb|AL592188.60|,18s.altHap.4870,AL592188.60,89.372,207,13,3,1,205,110172,110371,2.69e-63,252|
|18s.altHap.4870,gi|86475748|emb|AL592188.60|,18s.altHap.4870,AL592188.60,89.372,207,13,3,1,205,154144,154343,2.69e-63,252|
|18s.altHap.4870,gi|374532824|ref|NR_046257.1|;gi83321215|gb|DQ222453.1|,18s.altHap.4870,NR_046257.1,89.372,207,13,3,1,205,1095,1294,2.69e-63,252|
|18s.altHap.4870,gi|555853|gb|U13369.1|HSU13369,18s.altHap.4870,U13369.1,89.372,207,13,3,1,205,4751,4950,2.69e-63,252|
|18s.altHap.4870,gi|337377|gb|K03432.1|HUMRGEA,18s.altHap.4870,K03432.1,89.372,207,13,3,1,205,1236,1435,2.69e-63,252|
|18s.altHap.4870,gi|337376|gb|M10098.1|HUMRGE,18s.altHap.4870,M10098.1,89.372,207,13,3,1,205,1196,1395,2.69e-63,252|
|18s.altHap.4870,gi|36162|emb|X03205.1|;gi672884471|pdb|4UQ4|1;gi672884505|pdb|4UQ5|1;gi958159612|pdb|5A8L|B,18s.altHap.4870,X03205.1,89.372,207,13,3,1,205,1095,1294,2.69e-63,252|
|18s.altHap.4870,gi|1048348539|pdb|5K0Y|A,18s.altHap.4870,5K0Y_A,89.320,206,13,3,2,205,1005,1203,9.68e-63,250|
|18s.altHap.4870,gi|1032033222|gb|KX121618.1|,18s.altHap.4870,KX121618.1,89.320,206,13,3,2,205,1008,810,9.68e-63,250|
|18s.altHap.4870,gi|850480667|gb|CP011888.1|,18s.altHap.4870,CP011888.1,89.320,206,13,3,2,205,100836027,100835829,9.68e-63,250|
|18s.altHap.4870,gi|675970227|pdb|4W23|2;gi675970373|pdb|4W28|2,18s.altHap.4870,4W23_2,89.320,206,13,3,2,205,969,1167,9.68e-63,250|
|18s.altHap.4870,gi|577019615|ref|NR_046233.2|,18s.altHap.4870,NR_046233.2,89.320,206,12,4,2,205,5104,5301,9.68e-63,250|
|18s.altHap.4870,gi|570362313|gb|KF732650.1|,18s.altHap.4870,KF732650.1,89.320,206,13,3,2,205,43,241,9.68e-63,250|
|18s.altHap.4870,gi|475655804|gb|JX240411.1|,18s.altHap.4870,JX240411.1,89.320,206,13,3,2,205,245,443,9.68e-63,250|
|18s.altHap.4870,gi|511668571|tpg|BK000964.3|,18s.altHap.4870,BK000964.3,89.320,206,12,4,2,205,5104,5301,9.68e-63,250|
|18s.altHap.4870,gi|334360161|gb|JF894380.1|,18s.altHap.4870,JF894380.1,89.320,206,13,3,2,205,1096,1294,9.68e-63,250|
|18s.altHap.4870,gi|380750581|gb|JQ247698.1|,18s.altHap.4870,JQ247698.1,89.320,206,13,3,2,205,1098,1296,9.68e-63,250|
|18s.altHap.4870,gi|374532828|ref|NR_046261.1|,18s.altHap.4870,NR_046261.1,89.320,206,13,3,2,205,1424,1622,9.68e-63,250|
|18s.altHap.4870,gi|374088232|ref|NR_003278.3|,18s.altHap.4870,NR_003278.3,89.320,206,12,4,2,205,1097,1294,9.68e-63,250|
|18s.altHap.4870,gi|371913276|dbj|AK392932.1|,18s.altHap.4870,AK392932.1,89.320,206,13,3,2,205,185,383,9.68e-63,250|
|18s.altHap.4870,gi|371911277|dbj|AK400360.1|,18s.altHap.4870,AK400360.1,89.320,206,13,3,2,205,209,407,9.68e-63,250|
|18s.altHap.4870,gi|356582733|gb|JN163869.1|,18s.altHap.4870,JN163869.1,89.320,206,13,3,2,205,1081,1279,9.68e-63,250|
|18s.altHap.4870,gi|346716260|ref|NR_045132.1|,18s.altHap.4870,NR_045132.1,89.320,206,13,3,2,205,1104,1302,9.68e-63,250|
|18s.altHap.4870,gi|343965963|gb|JN412502.1|,18s.altHap.4870,JN412502.1,89.320,206,13,3,2,205,1096,1294,9.68e-63,250|
|18s.altHap.4870,gi|307829144|gb|GU372691.1|,18s.altHap.4870,GU372691.1,89.320,206,13,3,2,205,8108,8306,9.68e-63,250|
|18s.altHap.4870,gi|300422603|dbj|AB571241.1|,18s.altHap.4870,AB571241.1,89.320,206,13,3,2,205,734,932,9.68e-63,250|
|18s.altHap.4870,gi|198401753|gb|FJ172904.1|,18s.altHap.4870,FJ172904.1,89.320,206,13,3,2,205,245,443,9.68e-63,250|
|18s.altHap.4870,gi|194307128|gb|EU823286.1|,18s.altHap.4870,EU823286.1,89.320,206,13,3,2,205,1093,1291,9.68e-63,250|
|18s.altHap.4870,gi|164504489|gb|EU338395.1|,18s.altHap.4870,EU338395.1,89.320,206,13,3,2,205,253,451,9.68e-63,250|
|18s.altHap.4870,gi|164504488|gb|EU338394.1|,18s.altHap.4870,EU338394.1,89.320,206,13,3,2,205,81,279,9.68e-63,250|
|18s.altHap.4870,gi|162138805|dbj|AB370873.1|,18s.altHap.4870,AB370873.1,89.320,206,13,3,2,205,45,243,9.68e-63,250|
|18s.altHap.4870,gi|160336947|gb|EU236698.1|,18s.altHap.4870,EU236698.1,89.320,206,13,3,2,205,29,227,9.68e-63,250|
|18s.altHap.4870,gi|160336945|gb|EU236696.1|,18s.altHap.4870,EU236696.1,89.320,206,13,3,2,205,328,526,9.68e-63,250|
|18s.altHap.4870,gi|160336943|gb|EU236694.1|,18s.altHap.4870,EU236694.1,89.320,206,13,3,2,205,76,274,9.68e-63,250|
|18s.altHap.4870,gi|126512996|gb|EF426477.1|,18s.altHap.4870,EF426477.1,89.320,206,13,3,2,205,139,337,9.68e-63,250|
|18s.altHap.4870,gi|115553642|dbj|AK239829.1|,18s.altHap.4870,AK239829.1,89.320,206,13,3,2,205,1108,1306,9.68e-63,250|
|18s.altHap.4870,gi|89511625|emb|AJ875189.1|,18s.altHap.4870,AJ875189.1,89.320,206,13,3,2,205,420,222,9.68e-63,250|
|18s.altHap.4870,gi|78355009|gb|DQ235090.1|,18s.altHap.4870,DQ235090.1,89.320,206,13,3,2,205,12726,12924,9.68e-63,250|
|18s.altHap.4870,gi|374532772|ref|NR_046263.1|;gi37626311|gb|AY390526.1|,18s.altHap.4870,NR_046263.1,89.320,206,13,3,2,205,3131,3329,9.68e-63,250|
|18s.altHap.4870,gi|37956930|gb|AY265350.1|,18s.altHap.4870,AY265350.1,89.320,206,13,3,2,205,879,681,9.68e-63,250|
|18s.altHap.4870,gi|33943621|gb|AY350612.1|,18s.altHap.4870,AY350612.1,89.320,206,13,3,2,205,36,234,9.68e-63,250|
|18s.altHap.4870,gi|84579144|dbj|AB220473.1|,18s.altHap.4870,AB220473.1,89.320,206,13,3,2,205,897,1095,9.68e-63,250|
|18s.altHap.4870,gi|74211473|dbj|AK145507.1|,18s.altHap.4870,AK145507.1,89.320,206,13,3,2,205,498,696,9.68e-63,250|
|18s.altHap.4870,gi|74192232|dbj|AK158003.1|,18s.altHap.4870,AK158003.1,89.320,206,13,3,2,205,830,1028,9.68e-63,250|
|18s.altHap.4870,gi|66277499|gb|AY987422.1|,18s.altHap.4870,AY987422.1,89.320,206,13,3,2,205,221,419,9.68e-63,250|
|18s.altHap.4870,gi|27469381|gb|BC042411.1|,18s.altHap.4870,BC042411.1,89.320,206,13,3,2,205,165,363,9.68e-63,250|
|18s.altHap.4870,gi|18369732|emb|AJ311675.1|,18s.altHap.4870,AJ311675.1,89.320,206,13,3,2,205,1076,1274,9.68e-63,250|
|18s.altHap.4870,gi|18369731|emb|AJ311674.1|,18s.altHap.4870,AJ311674.1,89.320,206,13,3,2,205,1075,1273,9.68e-63,250|
|18s.altHap.4870,gi|374671780|ref|NR_046271.1|;gi18369730|emb|AJ311673.1|,18s.altHap.4870,NR_046271.1,89.320,206,13,3,2,205,1075,1273,9.68e-63,250|
|18s.altHap.4870,gi|374429560|ref|NR_046239.1|;gi2624399|emb|V01270.1|,18s.altHap.4870,NR_046239.1,89.320,206,13,3,2,205,1100,1298,9.68e-63,250|
|18s.altHap.4870,gi|1261918|emb|X82564.1|,18s.altHap.4870,X82564.1,89.320,206,12,4,2,205,10739,10936,9.68e-63,250|
|18s.altHap.4870,gi|206631|gb|M11188.1|RATRGEA,18s.altHap.4870,M11188.1,89.320,206,12,4,2,205,1109,1306,9.68e-63,250|
|18s.altHap.4870,gi|374429558|ref|NR_046237.1|;gi57149|emb|X01117.1|,18s.altHap.4870,NR_046237.1,89.320,206,13,3,2,205,1100,1298,9.68e-63,250|
|18s.altHap.4870,gi|53990|emb|X00686.1|,18s.altHap.4870,X00686.1,89.320,206,12,4,2,205,1097,1294,9.68e-63,250|
|18s.altHap.4870,gi|283837872|ref|NR_033238.1|;gi1707|emb|X06778.1|,18s.altHap.4870,NR_033238.1,89.320,206,13,3,2,205,1092,1290,9.68e-63,250|
|18s.altHap.4870,gi|485601472|pdb|3J3D|2,18s.altHap.4870,3J3D_2,88.889,207,14,3,1,205,1095,1294,3.48e-62,248|
|18s.altHap.4870,gi|398360298|gb|JX132355.1|,18s.altHap.4870,JX132355.1,89.268,205,13,3,1,203,472,669,3.48e-62,248|
|18s.altHap.4870,gi|398360276|gb|JX132333.1|;gi398360278|gb|JX132335.1|;gi398360279|gb|JX132336.1|;gi398360282|gb|JX132339.1|;gi398360287|gb|JX132344.1|;gi398360292|gb|JX132349.1|;gi398360299|gb|JX132356.1|;gi398360306|gb|JX132363.1|;gi398360311|gb|JX132368.1|;gi398360313|gb|JX132370.1|;gi398360319|gb|JX132376.1|;gi398360320|gb|JX132377.1|;gi398360321|gb|JX132378.1|;gi398360367|gb|JX132424.1|,18s.altHap.4870,JX132333.1,89.268,205,13,3,1,203,472,669,3.48e-62,248|
|18s.altHap.4870,gi|158121164|gb|EU164971.1|,18s.altHap.4870,EU164971.1,88.889,207,15,3,1,205,244,444,3.48e-62,248|